In [74]:
import pandas as pd
import openpyxl
import requests
import json

In [96]:

def translate(word):
    # 有道词典 api
    url = 'http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=null'
    # 传输的参数，其中 i 为需要翻译的内容
    key = {
        'type': "AUTO",
        'i': word,
        "doctype": "json",
        "version": "2.1",
        "keyfrom": "fanyi.web",
        "ue": "UTF-8",
        "action": "FY_BY_CLICKBUTTON",
        "typoResult": "true"
    }
    # key 这个字典为发送给有道词典服务器的内容
    response = requests.post(url, data=key)
    # 判断服务器是否相应成功
    response.raise_for_status()
    if response.status_code == 200:
        # 然后相应的结果
        return json.loads(response.text)["translateResult"][0][0]['tgt']
    else:
        print("有道词典调用失败")
        # 相应失败就返回空
        return None

In [99]:
cn = pd.read_excel('./统计数据.xlsx')
def raw_to_df(data):
    raw_list = data.splitlines(False)
    # header = raw_list[1].split()
    header = ['英文名', 'PWR', '平均抓位', '平均打出回合', '分发次数', '选择次数', '打出次数', '获胜次数(含未打出)', '获胜次数(仅统计打出)']
    df = pd.DataFrame(columns = header)
    for index in range(2, len(raw_list) - 1):
        l, r = 0, 0
        for i in range(len(raw_list[index]) - 1):
            if not raw_list[index][i].isalpha() and raw_list[index][i+1].isalpha() and l <= 0: l = i
            if raw_list[index][i].isalpha() and not raw_list[index][i+1].isalpha(): r = i
        tmp = [raw_list[index][l + 1 : r + 1]]
        tmp += raw_list[index][r + 2 :].split()
        df.loc[df.shape[0] - 1] = tmp
    df.reset_index(inplace=True, drop=True)
    res = df.merge(cn.loc[:, ['英文名', '中文名']], how = 'left', on= '英文名')
    # res['机翻'] = res['英文名'].apply(translate)
    return res
    # res.to_excel('./sample.xlsx', index = False)
    # res.tail(20)

In [71]:
all_arena_games = '''
            Card Name   PWR   ADP    APR  Deals  Drafted  Plays  Win-Hand  Win-Play
          Big Country  5.09  1.54   8.32    331      331    263       132       118   
     Assistant Tiller  4.43  1.28   3.37    319      319    297       105        99   
            Childless  4.42  2.16   3.48    330      330    258       123       102   
               Grocer  4.36  2.03   9.13    308      308    261       102        94   
   Carpenter's Parlor  4.04  1.23   4.13    336      336    320       100        95   
        Mining Hammer  3.98  2.97   6.65    327      327    272       107        91   
            Beanfield  3.79  3.80  10.18    328      328    256       108        87   
             Loam Pit  3.62  1.99   8.13    335      335    252       106        85   
      Sleeping Corner  3.54  1.90  10.90    331      331    219       113        82   
          Rammed Clay  3.51  1.64   4.51    330      330    321        84        81   
       Moldboard Plow  3.46  2.02   7.51    322      322    290        89        78   
                 Loom  3.42  2.20   9.82    313      313    260        89        75   
        House Steward  3.38  3.29   4.38    347      347    240       111        82   
         Mini Pasture  3.34  3.02   6.89    321      321    276        93        75   
     Shepherd's Crook  3.23  3.80   7.82    323      323    266        87        73   
   Small-scale Farmer  3.17  2.86   1.33    333      333    265        97        74   
     Strawberry Patch  3.17  3.48  12.79    342      342    178       115        76   
               Manger  3.16  4.03  11.61    326      326    201        97        72   
            Harpooner  3.05  2.25   3.94    323      323    268        84        69   
          Lumber Mill  2.99  3.52   6.67    334      334    246        91        70   
             Milk Jug  2.96  3.23   9.46    319      319    270        75        66   
         Butter Churn  2.94  3.13   7.99    340      340    288        81        70   
             Claypipe  2.92  5.07   8.31    323      323    192        98        66   
              Scholar  2.76  3.27   8.41    331      331    168       104        64   
      Seasonal Worker  2.73  2.22   4.33    319      319    227        88        61   
          Plow Driver  2.66  3.13   9.99    338      338    208        86        63   
    Storehouse Keeper  2.64  3.35   2.76    330      330    182       105        61   
               Priest  2.59  4.12   7.16    331      331    185       107        60   
      Threshing Board  2.55  4.43  10.71    319      319    191        88        57   
  Brushwood Collector  2.51  2.15   3.84    324      324    245        73        57   
             Pond Hut  2.48  3.77   6.92    311      311    205        82        54   
         Animal Tamer  2.46  4.07   8.88    308      308    197        79        53   
             Cottager  2.40  2.30   3.80    322      322    209        86        54   
       Roof Ballaster  2.38  3.86  11.99    324      324    147       115        54   
         Thick Forest  2.38  4.71   6.65    324      324    173        79        54   
           Manservant  2.29  3.96   8.72    312      312    148        95        50   
     Clay Hut Builder  2.26  3.76   7.08    335      335    190        87        53   
             Handplow  2.25  3.50   5.44    318      318    233        72        50   
              Bottles  2.17  3.22  10.52    329      329    197        82        50   
               Pastor  2.17  4.79   7.82    335      335    141        98        51   
    Master Bricklayer  2.16  4.45   9.28    324      324    130        93        49   
 Three-Field Rotation  2.16  4.63  10.56    337      337    195        86        51   
            Sack Cart  2.11  5.08   7.19    331      331    179        77        49   
         Bread Paddle  2.05  5.02   8.27    341      341    215        82        49   
          Mantlepiece  2.04  3.40   7.02    336      336    166        95        48   
                Canoe  2.01  3.02   5.61    334      334    251        66        47   
         Sheep Walker  2.00  3.40  10.89    321      321    164        92        45   
 Shifting Cultivation  1.94  4.61   8.90    338      338    240        77        46   
   Firewood Collector  1.89  4.12   4.11    355      355    178        86        47   
          Wood Cutter  1.88  2.44   2.04    334      334    280        58        44   
            Junk Room  1.84  4.71   5.79    334      334    222        69        43   
       Clearing Spade  1.84  4.92   9.22    326      326    163        77        42   
     Adoptive Parents  1.82  3.95   8.44    330      330    185        79        42   
       Organic Farmer  1.82  5.07  13.70    337      337    111       105        43   
          Conservator  1.79  3.05   9.42    312      312    198        63        39   
          Stonecutter  1.79  3.53   5.57    336      336    182        77        42   
          Herring Pot  1.79  4.24   6.48    319      319    187        69        40   
       Hard Porcelain  1.78  4.23   8.96    346      346    211        73        43   
            Carpenter  1.77  2.84   4.26    331      331    222        69        41   
           Consultant  1.69  4.70  12.51    347      347    119       105        41   
      Clay Embankment  1.58  5.41   9.70    343      343    152        87        38   
                Tutor  1.49  5.23   2.97    346      346    158        82        36   
                Lasso  1.48  4.45   9.43    329      329    166        74        34   
      Drinking Trough  1.48  5.09   9.88    329      329    163        66        34   
         Market Stall  1.46  5.95  10.65    332      332    148        78        34   
  Young Animal Market  1.45  5.70  11.37    355      355    164        87        36   
                Groom  1.44  5.02  10.58    348      348    106        90        35   
           Corn Scoop  1.41  3.96   7.06    344      344    246        60        34   
             Conjurer  1.40  3.99   3.81    347      347    166        86        34   
             Scullery  1.39  3.94   4.10    329      329    186        67        32   
        Acorns Basket  1.39  4.67  11.01    339      339    143        86        33   
          Greengrocer  1.38  3.73   7.52    342      342    216        58        33   
        Scythe Worker  1.30  4.75   9.24    329      329    133        74        30   
            Pitchfork  1.29  4.59   7.17    333      333    214        61        30   
         Hedge Keeper  1.16  3.56   7.33    321      321    181        59        26   
                Brook  1.13  4.57   5.58    341      341    149        59        27   
        Animal Dealer  1.12  3.90  10.21    331      331    135        65        26   
          Pig Breeder  1.11  5.38  12.42    335      335     99        93        26   
     Large Greenhouse  0.94  4.97   6.41    333      333    128        59        22   
       Dutch Windmill  0.91  5.66  11.59    331      331     79        70        21   
        Wool Blankets  0.91  6.05  12.60    328      328     68        86        21   
      Sheep Whisperer  0.90  5.14   3.95    317      317     96        57        20   
          Stone Tongs  0.86  4.72   8.43    332      332    180        51        20   
               Basket  0.82  6.10   8.06    330      330     90        72        19   
            Geologist  0.78  4.46   2.62    348      348    159        53        19   
             Lutenist  0.78  5.27   4.66    330      330    116        68        18   
        Frame Builder  0.73  4.48   7.70    332      332    128        52        17   
   Mushroom Collector  0.61  5.08   3.26    350      350    106        74        15   
      Oven Firing Boy  0.59  5.95  10.95    314      314     40        85        13   
        Cattle Feeder  0.52  4.85   9.38    332      332    120        50        12   
     Stable Architect  0.48  6.00  13.52    356      356     67        71        12   
          Paper Maker  0.45  5.83   5.91    321      321     74        77        10   
          Roughcaster  0.39  6.44   7.48    331      331     42        74         9   
         Wall Builder  0.26  6.13   5.55    326      326     56        62         6   '''



In [73]:
elo200_games = '''
            Card Name   PWR   ADP    APR  Deals  Drafted  Plays  Win-Hand  Win-Play
     Assistant Tiller  6.44  1.23   2.78     71       71     69        34        32 
          Big Country  5.10  1.33   8.15     70       70     61        27        25 
   Carpenter's Parlor  4.56  1.15   3.93     72       72     71        23        23 
             Pond Hut  3.94  3.09   6.66     58       58     41        21        16 
            Childless  3.93  1.86   3.17     80       80     64        27        22 
      Sleeping Corner  3.90  1.76  10.93     66       66     44        22        18 
               Grocer  3.90  1.89   9.09     66       66     58        19        18 
        Mining Hammer  3.76  2.70   7.09     76       76     68        23        20 
      Seasonal Worker  3.68  2.38   4.26     66       66     50        19        17 
          Rammed Clay  3.67  1.66   4.56     74       74     70        20        19 
     Shepherd's Crook  3.67  3.53   7.95     70       70     59        20        18 
   Small-scale Farmer  3.52  2.75   1.31     69       69     54        24        17 
          Lumber Mill  3.52  2.93   6.84     73       73     56        19        18 
             Loam Pit  3.42  1.80   8.17     71       71     53        23        17 
            Beanfield  3.35  3.43  10.34     81       81     65        26        19 
               Priest  3.33  3.67   6.69     73       73     45        22        17 
            Sack Cart  3.25  4.97   7.92     79       79     49        25        18 
  Brushwood Collector  3.20  2.10   3.46     67       67     50        19        15 
               Manger  3.13  3.77  11.16     73       73     50        20        16 
        House Steward  3.09  3.19   3.58     74       74     53        21        16 
             Milk Jug  3.06  3.59   9.33     70       70     60        16        15 
     Strawberry Patch  3.01  3.28  12.78     76       76     40        28        16 
         Thick Forest  2.96  4.70   6.81     82       82     47        24        17 
         Mini Pasture  2.90  2.92   7.60     64       64     57        16        13 
      Clay Embankment  2.71  5.24   9.43     79       79     40        26        15 
      Threshing Board  2.68  4.22  10.89     64       64     37        20        12 
       Roof Ballaster  2.64  3.89  11.70     65       65     33        17        12 
               Pastor  2.60  4.65   7.44     77       77     36        27        14 
     Clay Hut Builder  2.48  3.59   6.76     69       69     41        17        12 
       Hard Porcelain  2.48  4.17   8.91     75       75     45        18        13 
                Canoe  2.45  3.39   5.69     70       70     58        14        12 
 Three-Field Rotation  2.44  4.74  10.70     76       76     46        22        13 
             Cottager  2.38  2.06   3.14     72       72     50        16        12 
    Master Bricklayer  2.38  4.22   8.97     72       72     33        26        12 
             Handplow  2.35  3.36   4.95     73       73     56        16        12 
             Claypipe  2.34  4.61   8.85     61       61     39        16        10 
         Butter Churn  2.32  2.76   7.63     74       74     68        13        12 
              Bottles  2.32  3.24  10.33     74       74     51        17        12 
                 Loom  2.31  1.87   9.74     68       68     54        14        11 
         Sheep Walker  2.26  3.38  11.18     76       76     44        22        12 
          Mantlepiece  2.23  2.95   6.85     77       77     46        19        12 
         Hedge Keeper  2.21  4.14   6.61     71       71     41        22        11 
              Scholar  2.13  2.72   7.79     67       67     39        16        10 
          Conservator  2.04  3.36   9.98     70       70     42        14        10 
       Moldboard Plow  2.01  1.97   8.05     71       71     62        11        10 
    Storehouse Keeper  2.01  3.20   2.26     71       71     38        20        10 
           Consultant  2.01  4.66  12.46     71       71     24        20        10 
         Bread Paddle  1.99  4.94   8.55     79       79     53        19        11 
         Animal Tamer  1.98  4.29   8.61     65       65     38        15         9 
          Pig Breeder  1.95  5.45  12.41     66       66     22        25         9 
            Carpenter  1.94  3.16   4.51     81       81     47        23        11 
        Acorns Basket  1.93  5.01  11.59     74       74     32        22        10 
            Junk Room  1.92  4.79   6.27     67       67     51        13         9 
        Scythe Worker  1.86  4.75   9.69     77       77     39        21        10 
   Firewood Collector  1.81  4.00   3.44     87       87     41        21        11 
           Corn Scoop  1.74  4.24   7.24     74       74     46        20         9 
       Clearing Spade  1.74  4.74   8.65     74       74     37        20         9 
          Wood Cutter  1.63  2.53   1.51     79       79     63        14         9 
          Plow Driver  1.57  2.59   9.87     73       73     47        11         8 
          Stonecutter  1.54  3.20   5.71     65       65     42        10         7 
            Harpooner  1.52  2.15   3.75     66       66     53        11         7 
           Manservant  1.47  3.57   8.27     68       68     30        16         7 
          Herring Pot  1.47  4.78   6.83     78       78     35        18         8 
  Young Animal Market  1.41  5.77  11.54     81       81     35        20         8 
             Scullery  1.39  4.78   3.95     72       72     37        15         7 
      Drinking Trough  1.39  5.36   9.95     72       72     40        15         7 
         Market Stall  1.39  6.17  10.63     72       72     27        15         7 
     Large Greenhouse  1.33  4.95   6.11     75       75     35        10         7 
          Greengrocer  1.28  4.28   8.13     78       78     38        16         7 
     Adoptive Parents  1.26  3.88   8.11     68       68     37        14         6 
       Dutch Windmill  1.26  5.74  12.38     68       68     16        19         6 
        Animal Dealer  1.25  3.94  10.36     80       80     25        12         7 
             Lutenist  1.25  5.40   4.64     80       80     28        18         7 
            Pitchfork  1.16  5.22   7.90     74       74     42        18         6 
                Brook  1.13  5.04   6.40     76       76     30        10         6 
      Sheep Whisperer  1.08  5.36   4.44     66       66     18        13         5 
                Lasso  1.07  4.69   9.43     67       67     37        13         5 
               Basket  1.07  6.10   7.74     67       67     23        14         5 
 Shifting Cultivation  1.05  4.76   9.14     68       68     50        10         5 
             Conjurer  0.93  3.99   3.53     77       77     38        13         5 
                Groom  0.92  4.58  10.26     78       78     27        18         5 
        Wool Blankets  0.80  6.01  13.00     71       71     11        14         4 
                Tutor  0.74  5.25   3.42     77       77     26        16         4 
          Stone Tongs  0.69  5.21   8.56     62       62     32        10         3 
       Organic Farmer  0.63  4.79  13.78     68       68     23        16         3 
            Geologist  0.60  5.17   3.62     72       72     24        14         3 
   Mushroom Collector  0.57  5.29   3.76     75       75     25        13         3 
        Cattle Feeder  0.45  5.20  10.58     64       64     26        12         2 
      Oven Firing Boy  0.45  6.02  11.22     63       63      9        19         2 
          Roughcaster  0.36  6.48   7.12     79       79      8        22         2 
        Frame Builder  0.22  4.83   8.10     66       66     21        16         1 
     Stable Architect  0.20  5.92  13.60     72       72     15        15         1 
          Paper Maker  0.00  5.95   7.25     74       74     12        21         0 
         Wall Builder  0.00  6.49   6.50     77       77      6        17         0 
'''


In [79]:
incCD = '''
            Card Name   PWR   ADP    APR  Deals  Drafted  Plays  Win-Hand  Win-Play
      Begging Student  5.55  1.71   3.25    399      379    355       165       155
             Braggart  4.55  1.94  13.47    836      641    525       287       266
       Field Watchman  4.54  1.33   3.42    412      407    390       137       131
            Bookshelf  4.42  2.40   8.12    420      410    370       132       130
                Mason  4.39  3.28  12.25    407      366    285       141       125
        Lettuce Patch  4.32  3.47  10.10    446      424    362       147       135
             Bookcase  4.31  1.83   4.70    434      428    401       135       131
           Cultivator  4.12  2.41   5.25    402      392    350       120       116
            Furnisher  4.08  3.30   4.59    385      342    279       125       110
   Carpenter's Parlor  3.94  1.36   4.45   1786     1592   1451       529       493
               Patron  3.86  2.38   2.36    407      386    362       114       110
            Excavator  3.84  1.83   2.64    402      394    332       121       108
           Swing Plow  3.70  2.57   9.31    455      425    369       132       118
        Wood Barterer  3.70  2.63   5.02    394      376    319       113       102
  Wooden Hut Extender  3.69  2.47   5.34    376      350    313       110        97
            Reed Pond  3.63  3.75   8.68    417      379    295       122       106
          Rammed Clay  3.61  2.01   5.18   1783     1591   1518       470       450
        Chimney Sweep  3.60  4.00  11.98    369      348    247       118        93
     Assistant Tiller  3.57  1.57   3.95   1693     1511   1345       467       423
      Charcoal Burner  3.52  2.83   3.42    386      355    277       111        95
             Bonehead  3.44  3.34   3.70    394      370    309       109        95
               Grocer  3.42  2.62   9.54   1701     1460   1091       494       407
            Constable  3.42  4.13   6.30    397      318    224       108        95
        Milking Stool  3.38  3.96  10.92    448      384    294       124       106
              Caravan  3.32  1.56   5.50    873      694    567       233       203
        Clay Supports  3.28  2.21   6.63    462      433    361       129       106
         Field Fences  3.28  3.11  11.75    444      420    332       114       102
           Bricklayer  3.28  3.17   5.60    375      340    267        95        86
        Lantern House  3.27  2.66   1.81    467      387    308       124       107
         Writing Desk  3.26  3.30   7.55    451      406    347       113       103
             Loam Pit  3.20  2.20   8.52   1809     1605   1207       489       405
                 Loom  3.20  3.01  10.05   1777     1488   1127       481       398
      Potter Ceramics  3.18  3.19   5.64    462      409    362       115       103
            Pet Lover  3.15  2.77   5.87    403      359    294        99        89
  Half-Timbered House  3.11  3.50  13.28    446      399    224       121        97
      Education Bonus  3.11  3.61   6.76    436      380    307       112        95
         Water Worker  3.08  2.75   3.56    408      381    295       105        88
          Big Country  3.06  2.77   9.14   1789     1338    774       504       383
            Wood Cart  3.03  2.74   8.10    485      467    360       129       103
       Master Builder  2.97  4.78  13.11    414      300    214       102        86
        Mining Hammer  2.94  3.30   7.23   1773     1487   1165       432       365
            Harpooner  2.90  2.18   3.67   1737     1513   1214       424       353
       Private Forest  2.82  3.24   3.42    435      367    269       102        86
          Wood Expert  2.79  3.46   4.79    374      325    248        93        73
      Seasonal Worker  2.78  2.19   4.47   1721     1538   1139       445       335
       Moldboard Plow  2.78  2.25   7.62   1786     1583   1405       385       347
            Blueprint  2.77  2.81   6.63    449      423    326       101        87
        Carrot Museum  2.74  3.11   7.22    417      357    249       107        80
               Porter  2.74  3.35   4.43    406      339    265        90        78
      Furniture Maker  2.70  4.13   3.33    386      318    256        85        73
         Mini Pasture  2.69  3.17   7.46   1739     1487   1250       396       328
            Beanfield  2.66  4.15  10.47   1785     1409   1034       426       332
           Feed Fence  2.65  4.17   7.97    448      360    296        96        83
   Skillful Renovator  2.63  4.40   8.74    440      334    218       101        81
        House Steward  2.62  3.70   4.92   1762     1393    928       451       323
       Turnwrest Plow  2.59  2.97   9.00    458      409    345        96        83
  Brushwood Collector  2.52  2.23   4.36   1722     1521   1146       398       304
          Conservator  2.52  2.82   9.85   1719     1502   1107       385       303
   Small-scale Farmer  2.51  3.03   1.58   1715     1329   1070       366       301
       Wood Collector  2.51  3.66   4.32    376      336    253        82        66
           Clay Firer  2.48  4.58   9.34    386      308    167        95        67
          Foreign Aid  2.46  4.38   5.09    448      260    201        87        77
     Winter Caretaker  2.43  4.54   8.75    399      292    212        78        68
     Reed-Hatted Toad  2.36  3.07   1.92    430      376    292        99        71
             Milk Jug  2.36  3.40   9.69   1774     1483   1204       357       293
          Clay Supply  2.35  3.88   5.14    455      404    325        96        75
  Schnapps Distillery  2.32  4.01  11.34    456      385    250       101        74
          Supply Boat  2.31  4.26   8.42    421      329    261        77        68
      Sleeping Corner  2.29  2.59  11.10   1773     1513    817       410       284
         Feed Pellets  2.29  4.75   8.00    462      346    285        84        74
  Straw-Thatched Roof  2.28  2.92   8.43    451      397    271        90        72
          Lumber Mill  2.28  3.63   6.55   1802     1387    912       391       287
           Wood Field  2.28  3.88   8.98    295      246    191        54        47
                Canoe  2.27  2.55   5.51   1806     1569   1168       369       287
              Trident  2.27  4.35   9.69    478      396    303        94        76
    Master Bricklayer  2.26  4.04   9.21   1721     1353    661       432       272
          Wood Cutter  2.25  2.31   2.45   1743     1551   1263       331       275
         Game Catcher  2.24  4.82  13.60    402      340    228        82        63
         Chicken Coop  2.23  3.83   6.33    435      366    252        89        68
         Butter Churn  2.21  3.73   7.88   1755     1359   1017       344       271
          Bellfounder  2.21  3.97   6.57    388      268    164        83        60
                Hutch  2.16  3.95   7.88    443      394    269        89        67
             Cottager  2.15  2.61   4.52   1742     1491    966       413       262
     Small Greenhouse  2.15  4.39   7.18    459      336    213        99        69
     Horse-Drawn Boat  2.14  4.22   8.24    414      333    208        93        62
       Roof Ballaster  2.12  3.94  11.90   1712     1357    669       453       254
              Bottles  2.09  3.52  10.08   1733     1424    812       383       253
              Scholar  2.09  3.76   8.57   1720     1259    664       413       252
       Recycled Brick  2.09  5.10  10.92    445      296    181        92        65
             Pond Hut  2.08  3.75   6.88   1768     1399    903       378       258
       Cross-Cut Wood  2.08  4.83  11.31    453      307    197        93        66
     Strawberry Patch  2.07  4.02  12.73   1823     1404    665       424       264
             Stockman  2.05  4.77  10.80    383      294    191        70        55
            Childless  2.03  3.15   3.90   1742     1140    696       349       248
           Plow Maker  2.02  3.55   8.31    431      370    287        78        61
         Clay Carrier  2.00  5.05   7.80    393      273    158        76        55
               Manger  1.98  4.31  11.68   1837     1172    676       346       255
     Shepherd's Crook  1.97  3.62   7.88   1758     1340   1049       308       243
   Schnapps Distiller  1.96  3.89   7.79    386      328    217        71        53
          Plow Driver  1.93  3.63  10.06   1737     1274    685       368       235
     Potato Harvester  1.93  4.44   8.37    399      318    231        73        54
             Layabout  1.92  4.33   9.33    410      333    216        84        55
     Clay Deliveryman  1.91  3.93   5.60    426      358    223        95        57
            Junk Room  1.89  3.84   5.55   1793     1381   1022       310       237
    Storehouse Keeper  1.87  3.23   3.32   1741     1393    781       368       228
         Animal Tamer  1.87  3.83   8.34   1710     1442    845       340       224
             Claypipe  1.87  4.59   8.09   1745     1186    738       336       229
        Carriage Trip  1.87  4.68   9.64    443      301    224        73        58
      Threshing Board  1.86  4.23  10.36   1733     1301    805       332       226
     Wildlife Reserve  1.86  4.39  10.56    438      356    225        84        57
       Hard Porcelain  1.85  3.98   9.32   1817     1401    877       351       235
           Manservant  1.81  3.94   8.76   1721     1269    644       371       218
             Handplow  1.78  3.43   5.75   1799     1408    958       311       224
                Flail  1.77  4.47   7.09    445      333    267        63        55
           Farm Store  1.75  4.08   7.78    448      335    217        82        55
           Stone Cart  1.74  3.99   6.26    451      353    209        85        55
                Baker  1.74  4.33   9.72    434      347    211        79        53
 Three-Field Rotation  1.71  4.40  10.56   1774     1328    813       311       212
              Plowman  1.70  3.81   3.61    404      305    222        64        48
   Domestician Expert  1.70  4.86   7.82    421      257    145        72        50
         Market Crier  1.69  3.79   7.35    389      314    252        56        46
          Herring Pot  1.68  3.46   6.13   1778     1407    929       310       209
         Clay Kneader  1.68  4.24   5.48    382      313    226        57        45
            Carpenter  1.64  2.65   4.61   1757     1500   1085       308       202
       Truffle Slicer  1.63  4.68  10.40    455      319    192        68        52
          Stonecutter  1.61  3.55   6.38   1736     1442    760       344       196
     Clay Hut Builder  1.60  3.83   7.54   1740     1368    713       326       195
         Bread Paddle  1.60  4.54   8.36   1772     1222    802       290       199
         Stork's Nest  1.59  4.00   9.64    448      326    163        86        50
              Granary  1.59  4.80   7.97    440      264    162        69        49
         Hedge Keeper  1.58  3.20   8.19   1758     1549   1031       309       194
          Greengrocer  1.57  2.98   6.80   1724     1498   1102       268       190
     Adoptive Parents  1.55  3.97   8.13   1747     1333    742       330       189
              Cowherd  1.55  5.12  10.70    387      258    104        69        42
          Seed Trader  1.54  5.04  10.26    380      186    108        56        41
        Animal Dealer  1.53  3.67   9.78   1720     1441    659       338       184
        Steam Machine  1.53  4.20   9.83    430      329    190        82        46
               Priest  1.53  4.35   7.26   1759     1105    591       335       189
        Stable Master  1.53  4.97   8.27    401      269    173        63        43
                Tutor  1.52  4.41   2.68   1743     1193    770       290       186
           Corn Scoop  1.50  3.12   6.49   1810     1511   1175       264       190
          Stone Tongs  1.49  4.14   8.42   1770     1363    826       291       185
             Thresher  1.48  4.24   7.65    397      283    179        62        41
 Shifting Cultivation  1.47  4.21   8.98   1752     1289    924       282       180
           Sheep Well  1.47  5.09  11.57    456      289    164        66        47
       Clay Plasterer  1.45  4.03   7.22    434      352    243        63        44
          Fishing Net  1.44  3.97   7.42    416      320    219        60        42
            Puppeteer  1.42  4.28   3.65    392      293    182        61        39
       Organic Farmer  1.39  5.01  13.36   1732     1140    395       351       168
            Sack Cart  1.37  4.18   7.19   1788     1226    655       258       171
             Scullery  1.36  3.40   4.29   1776     1366    827       287       169
                Lasso  1.36  4.06   9.58   1756     1324    730       309       167
          Seed Seller  1.35  3.99   6.51    392      310    222        49        37
          Home Brewer  1.35  4.55   8.54    402      311    178        53        38
         Thick Forest  1.34  4.56   7.19   1781     1225    599       281       167
       Stone Clearing  1.34  5.15  10.55    427      289    166        66        40
         Sheep Walker  1.33  3.79  10.61   1717     1346    608       316       160
        Acorns Basket  1.24  4.51  11.04   1802     1319    591       319       157
      Building Tycoon  1.22  3.99   5.75    421      316    154        75        36
           Millwright  1.20  4.57   8.32    392      272    167        52        33
       Hunting Trophy  1.20  4.88  11.58    298      193     92        47        25
           Consultant  1.18  4.55  11.71   1814     1308    541       333       150
          Mantlepiece  1.15  4.12   7.31   1761     1064    452       320       142
               Stable  1.15  5.13   9.51    446      284    206        48        36
        Milking Place  1.14  4.61   9.22    489      303    170        62        39
     Livestock Feeder  1.14  4.91   9.11    401      262    138        57        32
            Pitchfork  1.13  4.03   7.26   1797     1335    881       216       142
               Pastor  1.10  4.71   8.17   1734     1082    442       301       134
         Stone Carver  1.08  4.84   7.12    409      231    116        54        31
        Pattern Maker  1.07  5.09   9.04    375      202     89        43        28
   Firewood Collector  1.06  3.84   4.60   1774     1357    676       258       132
   Mushroom Collector  1.05  3.96   3.32   1762     1289    561       285       130
                Groom  1.05  4.84  10.51   1780     1179    407       304       131
                Brook  1.04  3.96   5.55   1802     1304    583       262       131
        Scythe Worker  1.03  4.37   8.19   1768     1235    580       256       127
            Collector  1.02  4.32   8.91    407      224    128        47        29
      Drinking Trough  0.99  4.31   9.77   1810     1249    680       224       125
       Clearing Spade  0.98  4.58   9.43   1810     1183    598       250       124
             Merchant  0.92  4.44   6.61    418      289    140        61        27
          Paper Maker  0.92  4.91   5.41   1697     1095    494       258       109
            Geologist  0.91  3.73   3.01   1791     1401    739       232       114
     Large Greenhouse  0.91  4.17   6.57   1817     1245    510       242       116
       Fodder Planter  0.91  5.12  11.72    251      146     67        24        16
      Clay Embankment  0.90  5.08   9.35   1770     1075    489       253       112
    Resource Recycler  0.90  5.16  10.95    398      233    105        59        25
           Transactor  0.89  5.15  13.32    385      198     88        47        24
          Cube Cutter  0.88  5.38   9.21    404      197    103        43        25
          Hide Farmer  0.88  5.42  13.30    275      145     64        34        17
             Conjurer  0.84  3.79   4.09   1751     1254    657       251       103
        Cattle Feeder  0.84  4.13   9.14   1737     1334    567       239       102
           Game Trade  0.82  5.21  12.49    452      242    105        60        26
        Frame Builder  0.77  4.26   8.14   1743     1266    502       236        94
            Muck Rake  0.77  5.47  11.41    443      173    102        34        24
          Pig Breeder  0.74  4.96  11.92   1722     1142    342       281        89
         Market Stall  0.74  5.30  10.23   1780     1000    475       203        92
            Sculpture  0.72  5.12  12.00    473      264     47        62        24
            Freemason  0.71  4.59   6.34    402      181     89        39        20
         Cattle Buyer  0.71  5.21  10.55    381      227     75        51        19
      Sheep Whisperer  0.70  4.38   4.10   1717     1164    467       214        84
  Young Animal Market  0.70  5.17  11.22   1828     1084    507       211        90
             Lutenist  0.66  4.61   4.83   1753     1188    476       233        81
               Basket  0.61  4.97   7.05   1807      983    382       195        77
           Stablehand  0.60  4.77   9.06    381      230    112        36        16
  Reed Roof Renovator  0.59  5.55   8.06    385      185     79        35        16
       Dutch Windmill  0.50  4.95  11.65   1769     1055    255       195        62
   Interior Decorator  0.50  5.26   6.21    400      145     53        27        14
         Wall Builder  0.49  4.76   4.98   1735     1070    371       186        60
     Stable Architect  0.44  5.53  13.22   1751      977    255       189        54
      Oven Firing Boy  0.40  4.94   9.44   1720     1053    213       201        48
         Bean Counter  0.40  4.97   5.36    397      179     78        31        11
        Wool Blankets  0.35  5.67  12.54   1773      792    173       183        44
       Dwelling Mound  0.34  4.88   2.32    461      164     81        27        11
      Ravenous Hunger  0.34  4.95   9.98    469      195     82        32        11
     Cattle Whisperer  0.31  5.01   6.19    411      175     63        25         9
          Roughcaster  0.26  5.59   8.04   1736      899    217       171        32
          Sugar Baker  0.24  5.16   8.67    424      151     39        26         7
'''

In [80]:
elo200CD = '''
            Card Name   PWR   ADP    APR  Deals  Drafted  Plays  Win-Hand  Win-Play
              Caravan  5.71  1.20   4.96     25       25     25        10        10
   Carpenter's Parlor  5.17  1.21   4.20    163      163    156        61        59
        Lettuce Patch  5.01  3.24   9.52     77       75     66        28        27
            Bookshelf  4.97  2.10   8.29     69       69     66        24        24
       Field Watchman  4.95  1.21   3.45     75       75     71        27        26
          Big Country  4.73  2.03   8.55    151      142    108        59        50
     Assistant Tiller  4.60  1.36   3.91    146      146    137        51        47
               Patron  4.52  2.25   2.13     60       57     55        19        19
             Bonehead  4.45  3.27   4.33     61       59     52        20        19
      Begging Student  4.31  1.53   3.49     73       73     70        22        22
         Writing Desk  4.20  3.00   7.31     68       67     61        20        20
        Wood Barterer  4.11  2.42   4.80     66       66     61        19        19
     Reed-Hatted Toad  3.98  2.80   1.95     61       56     43        23        17
          Rammed Clay  3.95  1.76   5.06    159      157    150        46        44
     Winter Caretaker  3.90  4.40   8.77     66       52     43        19        18
            Furnisher  3.82  2.86   4.69     71       70     62        21        19
      Furniture Maker  3.79  3.72   3.22     64       53     40        20        17
             Braggart  3.76  1.16  13.53     19       19     19         5         5
        Mining Hammer  3.76  3.20   7.14    167      160    139        49        44
        Milking Stool  3.72  3.96  11.37     73       69     54        21        19
               Grocer  3.70  2.15   9.17    166      166    145        47        43
             Loam Pit  3.69  2.05   8.36    155      155    128        48        40
       Wood Collector  3.64  3.86   4.32     55       51     40        14        14
            Reed Pond  3.63  3.51   9.04     63       57     45        18        16
      Charcoal Burner  3.62  2.45   3.53     67       65     53        18        17
      Potter Ceramics  3.62  3.33   5.16     71       69     61        20        18
             Bookcase  3.52  1.71   4.71     73       73     66        19        18
        Lantern House  3.52  2.22   1.63     73       69     59        19        18
                Mason  3.52  3.16  12.56     65       62     52        17        16
           Cultivator  3.51  2.43   5.24     61       61     55        16        15
  Brushwood Collector  3.40  2.39   4.06    147      145    109        45        35
           Swing Plow  3.38  2.60   9.15     76       70     59        19        18
          Conservator  3.38  3.29  10.07    152      148    104        41        36
        House Steward  3.35  3.61   4.59    162      157    119        45        38
  Wooden Hut Extender  3.34  2.43   5.05     47       44     41        12        11
            Constable  3.33  4.02   5.61     73       61     44        21        17
         Water Worker  3.28  3.11   3.67     61       54     43        19        14
           Clay Firer  3.28  4.51  10.03     61       49     35        14        14
       Recycled Brick  3.24  5.04  10.72     75       55     39        23        17
      Sleeping Corner  3.20  2.23  10.92    152      149    100        43        34
       Roof Ballaster  3.17  3.70  11.75    144      140     80        42        32
            Beanfield  3.16  4.09  10.58    172      165    131        49        38
         Mini Pasture  3.15  3.40   8.33    145      141    124        37        32
       Master Builder  3.13  5.02  13.17     73       55     42        17        16
      Seasonal Worker  3.11  2.37   4.62    147      147    109        40        32
       Private Forest  3.11  3.54   3.15     69       57     41        18        15
   Small-scale Farmer  3.10  2.89   1.35    143      127    106        39        31
            Childless  3.09  2.31   3.23    162      147    112        41        35
        Chimney Sweep  3.02  4.02  12.28     52       51     39        14        11
     Strawberry Patch  2.98  3.87  12.85    163      159     89        53        34
                 Loom  2.91  2.72  10.26    167      163    133        40        34
             Milk Jug  2.90  3.73   9.64    143      135    115        34        29
   Skillful Renovator  2.90  3.92   9.16     64       49     38        15        13
  Straw-Thatched Roof  2.89  2.91   7.86     79       76     57        19        16
            Excavator  2.86  1.81   2.57     70       69     61        15        14
            Pet Lover  2.86  2.67   5.57     65       61     56        14        13
                Flail  2.86  4.73   7.55     75       56     42        18        15
             Pond Hut  2.82  3.64   6.92    142      128     93        38        28
               Priest  2.82  4.09   6.73    152      135     85        37        30
     Small Greenhouse  2.82  4.20   6.52     76       61     46        19        15
      Education Bonus  2.81  3.73   6.43     61       60     51        15        12
             Stockman  2.81  5.18  11.50     56       44     28        14        11
          Foreign Aid  2.78  4.37   4.84     72       46     37        14        14
            Blueprint  2.73  2.78   6.49     68       67     51        14        13
          Lumber Mill  2.72  3.42   6.32    168      146    114        36        32
         Clay Carrier  2.72  5.00   6.73     63       45     26        17        12
        Clay Supports  2.69  1.91   6.66     69       68     59        15        13
        Carrot Museum  2.66  2.64   7.26     59       56     47        12        11
  Half-Timbered House  2.65  3.47  13.37     70       66     38        14        13
                Hutch  2.63  4.15   8.26     76       67     46        16        14
          Bellfounder  2.62  4.02   6.47     60       47     30        12        11
       Moldboard Plow  2.60  2.40   8.04    154      152    137        29        28
            Wood Cart  2.54  2.58   8.28     90       89     74        19        16
      Threshing Board  2.49  4.63  10.20    155      126     81        38        27
       Turnwrest Plow  2.48  3.18   9.60     75       67     60        15        13
     Horse-Drawn Boat  2.48  4.44   8.07     75       64     42        17        13
       Cross-Cut Wood  2.48  4.57  11.26     69       53     39        15        12
  Schnapps Distillery  2.48  4.61  11.54     69       61     37        19        12
           Wood Field  2.44  4.13   8.72     41       30     25         7         7
     Clay Deliveryman  2.38  4.17   6.26     72       66     43        17        12
     Shepherd's Crook  2.37  4.04   7.97    151      127    101        32        25
           Bricklayer  2.35  3.06   5.29     67       65     49        12        11
    Master Bricklayer  2.35  4.17   9.16    146      140     69        44        24
          Wood Cutter  2.32  2.76   2.15    160      156    121        37        26
       Hard Porcelain  2.32  4.20   9.09    160      144     96        32        26
            Carpenter  2.31  3.02   4.92    161      157    103        41        26
         Market Crier  2.30  4.15   7.30     56       41     33        10         9
           Plow Maker  2.29  3.59   9.26     75       63     50        13        12
         Feed Pellets  2.29  4.91   8.05     75       55     44        13        12
         Hedge Keeper  2.28  4.20   8.23    169      159    105        44        27
         Clay Kneader  2.28  4.47   6.00     69       58     45        12        11
               Manger  2.27  4.10  11.39    170      116     79        35        27
              Bottles  2.26  3.58  10.08    164      152    102        39        26
    Storehouse Keeper  2.26  3.58   3.19    152      141     81        40        24
               Pastor  2.25  4.84   7.84    165      135     75        43        26
         Field Fences  2.20  2.90  12.20     65       62     51        12        10
         Animal Tamer  2.19  4.42   9.06    150      140     80        34        23
                Canoe  2.16  3.08   5.64    172      167    138        32        26
       Clay Plasterer  2.16  4.29   7.03     66       58     36        13        10
             Handplow  2.14  3.71   5.46    167      154    114        34        25
          Clay Supply  2.14  3.96   5.41     80       73     61        13        12
       Fodder Planter  2.13  5.44  11.22     47       25     18         8         7
              Trident  2.12  4.61   9.63     74       62     52        13        11
         Butter Churn  2.10  3.54   7.81    163      151    124        27        24
              Plowman  2.10  3.89   4.21     68       55     42        11        10
         Bread Paddle  2.10  4.81   8.20    170      142     95        37        25
         Thick Forest  2.09  5.00   7.06    171      137     77        37        25
     Clay Hut Builder  2.08  3.88   7.27    151      142     85        31        22
          Seed Trader  2.07  5.56  10.50     62       32     20        11         9
           Stone Cart  2.06  4.17   6.73     90       69     45        20        13
             Cottager  2.04  2.33   3.96    168      165    117        36        24
         Stork's Nest  2.04  3.89   9.26     70       54     31        15        10
           Feed Fence  2.04  4.52   8.13     77       63     53        13        11
        Pattern Maker  2.04  5.27   8.95     56       33     21        10         8
        Animal Dealer  2.01  4.05  10.00    149      148     58        32        21
     Potato Harvester  2.01  4.78   8.73     78       64     44        17        11
              Scholar  1.99  3.49   8.20    151      135     79        35        21
          Home Brewer  1.96  5.22   8.78     51       37     23         9         7
           Farm Store  1.95  3.76   6.39     66       50     33        14         9
          Greengrocer  1.95  3.92   7.96    161      150     97        33        22
         Sheep Walker  1.94  3.73  10.36    155      146     88        37        21
          Plow Driver  1.90  3.18   9.86    150      138     86        28        20
        Steam Machine  1.89  4.07   9.79     68       57     33        13         9
             Layabout  1.89  4.38   8.66     68       53     38        12         9
            Sack Cart  1.89  4.64   7.67    174      135     86        34        23
              Cowherd  1.89  5.85  10.77     53       27     13         9         7
        Carriage Trip  1.86  5.11  11.00     77       55     37        15        10
            Harpooner  1.79  2.42   3.67    152      149    123        27        19
               Porter  1.79  3.16   4.24     64       55     42        10         8
        Milking Place  1.79  4.64   8.37     80       56     30        15        10
          Cube Cutter  1.76  5.39  11.05     65       33     20        11         8
             Scullery  1.75  4.49   4.09    163      135     76        36        20
        Acorns Basket  1.75  5.22  11.22    171      141     73        43        21
      Clay Embankment  1.69  5.42   9.74    177      129     61        35        21
            Junk Room  1.68  4.46   5.35    162      131    113        24        19
 Three-Field Rotation  1.68  4.84  10.84    170      154     95        34        20
                Baker  1.66  4.34   9.93     69       61     41        13         8
       Truffle Slicer  1.66  4.81  10.51     86       63     39        11        10
          Wood Expert  1.64  2.87   4.58     61       53     43         9         7
          Stonecutter  1.63  3.57   6.32    149      143     91        25        17
         Chicken Coop  1.61  3.68   6.28     71       60     43         9         8
          Supply Boat  1.61  4.69   8.02     80       68     56        10         9
        Scythe Worker  1.58  4.96   9.82    154      122     62        31        17
          Pig Breeder  1.58  5.44  12.38    154      124     47        40        17
            Puppeteer  1.57  4.07   3.51     73       57     39        12         8
          Mantlepiece  1.56  3.45   6.88    156      121     72        29        17
 Shifting Cultivation  1.55  4.90   9.67    157      130     96        29        17
             Claypipe  1.52  4.59   8.26    150      128     88        27        16
     Wildlife Reserve  1.47  4.47  10.48     68       58     40         7         7
           Corn Scoop  1.45  3.97   6.79    168      152    102        36        17
          Fishing Net  1.45  4.32   7.30     69       57     37        11         7
           Consultant  1.45  4.88  12.92    167      133     52        32        17
   Firewood Collector  1.44  4.07   3.91    179      152     82        34        18
         Game Catcher  1.43  5.42  13.61     60       52     41         9         6
       Clearing Spade  1.42  4.98   8.63    171      130     67        33        17
       Organic Farmer  1.40  4.92  13.36    143      127     53        31        14
     Adoptive Parents  1.39  4.20   7.95    144      128     76        26        14
         Stone Carver  1.39  5.58   7.19     82       43     21        12         8
   Schnapps Distiller  1.36  3.98   7.65     63       54     34         9         6
        Stable Master  1.33  5.37   8.45     75       43     31         9         7
          Paper Maker  1.33  5.40   6.57    140      115     42        35        13
     Large Greenhouse  1.32  4.79   6.28    173      135     71        23        16
          Herring Pot  1.31  4.60   6.41    163      146     82        29        15
            Pitchfork  1.31  5.04   7.70    164      133     83        28        15
           Manservant  1.30  3.98   8.38    154      131     60        29        14
               Stable  1.30  5.67  10.07     66       39     29         6         6
                Lasso  1.27  4.73   9.44    157      133     75        27        14
                Tutor  1.27  4.91   2.95    146      123     58        30        13
              Granary  1.19  4.93   8.55     60       27     20         5         5
     Livestock Feeder  1.19  5.26   9.88     72       50     25        11         6
   Domestician Expert  1.17  4.69   8.78     61       39     18         8         5
          Hide Farmer  1.14  5.48  13.42     50       31     12         7         4
  Young Animal Market  1.09  5.81  11.86    184      119     51        32        14
   Mushroom Collector  1.07  4.68   4.28    160      136     65        26        12
           Transactor  1.07  5.59  13.56     67       34     18        10         5
           Sheep Well  1.06  5.22  11.97     81       49     29        11         6
             Merchant  1.05  4.37   6.40     68       43     20        13         5
    Resource Recycler  1.05  5.27  10.30     68       41     20        12         5
            Sculpture  1.03  5.20  12.00     83       49     11        16         6
            Collector  1.02  4.40   8.36     70       45     22        10         5
      Building Tycoon  1.01  3.98   5.07     71       47     27        10         5
          Stone Tongs  1.01  4.95   8.51    142      126     71        23        10
       Stone Clearing  1.01  5.51  10.88     71       47     25        12         5
           Millwright  1.00  4.86   8.08     57       36     25         8         4
                Groom  0.99  4.61  10.48    159      120     46        32        11
         Market Stall  0.92  6.14  10.60    155      102     43        19        10
      Drinking Trough  0.91  5.51  10.09    173      116     65        22        11
       Hunting Trophy  0.89  4.45  11.72     48       31     18         7         3
             Conjurer  0.85  4.04   4.11    152      122     65        23         9
          Seed Seller  0.85  4.66   7.03     67       53     36         6         4
                Brook  0.83  4.91   6.64    173      128     59        18        10
      Oven Firing Boy  0.79  5.93  10.36    145       94     25        30         8
        Cattle Feeder  0.77  5.07  10.20    148      127     60        26         8
      Sheep Whisperer  0.72  5.36   5.00    138       96     30        20         7
       Dutch Windmill  0.72  5.60  12.22    159      106     27        23         8
               Basket  0.71  5.96   7.38    161       91     34        20         8
         Cattle Buyer  0.68  5.56  12.10     63       34     10         7         3
             Lutenist  0.66  5.48   4.91    152      116     45        23         7
           Stablehand  0.63  5.76  10.38     68       29      8         9         3
            Freemason  0.58  4.78   7.25     74       32     16         5         3
           Game Trade  0.57  5.68  12.43     75       31     14         8         3
       Dwelling Mound  0.54  4.66   2.35     80       35     20         5         3
        Frame Builder  0.54  4.90   8.33    158      134     55        26         6
     Cattle Whisperer  0.48  5.55   6.40     59       20      5         5         2
     Stable Architect  0.47  6.00  13.45    151      109     33        23         5
        Wool Blankets  0.47  6.04  13.00    153       80     15        18         5
          Roughcaster  0.46  6.17   8.04    155      113     23        30         5
             Thresher  0.45  4.60   8.44     63       47     34         5         2
            Geologist  0.45  5.04   2.92    158      132     59        20         5
   Interior Decorator  0.41  5.42   7.00     69       19      5         4         2
         Bean Counter  0.26  5.50   6.75     56       18      8         2         1
  Reed Roof Renovator  0.23  5.96   7.36     63       25     11         4         1
            Muck Rake  0.23  6.04  11.85     63       24     13         3         1
          Sugar Baker  0.21  6.10   8.50     67       20      6         4         1
      Ravenous Hunger  0.19  5.46  10.43     76       24      7         7         1
         Wall Builder  0.09  6.18   5.65    168      109     20        20         1
'''

In [81]:
elo100CD = '''
                Card Name   PWR   ADP    APR  Deals  Drafted  Plays  Win-Hand  Win-Play
0         Begging Student  5.44  1.52   3.25    239      233    221        95        91
1                Braggart  5.14  1.29  13.57     75       73     68        27        27
2                 Caravan  4.70  1.18   4.85     82       82     79        27        27
3          Field Watchman  4.65  1.29   3.37    255      255    246        87        83
4                Bookcase  4.48  1.80   4.52    252      251    235        81        79
5               Bookshelf  4.34  2.29   8.21    250      248    227        78        76
6                    Loom  4.30  3.32  10.13    266      256    218        85        80
7           Lettuce Patch  4.27  3.39  10.14    254      243    208        81        76
8                Bonehead  4.19  3.21   3.88    225      219    189        74        66
9              Cultivator  4.08  2.40   5.15    231      229    205        69        66
10               Loam Pit  4.02  2.30   8.66    263      263    220        81        74
11                  Mason  4.00  3.19  12.58    232      214    171        73        65
12     Carpenter's Parlor  3.87  1.28   4.44    266      265    248        78        72
13          Wood Barterer  3.86  2.67   4.96    237      233    197        70        64
14                 Patron  3.82  2.30   2.44    243      232    219        66        65
15    Wooden Hut Extender  3.72  2.47   5.45    223      210    194        63        58
16              Excavator  3.66  1.87   2.56    238      236    202        69        61
17             Swing Plow  3.62  2.52   9.21    272      254    225        74        69
18            Rammed Clay  3.61  2.06   6.10    253      251    240        65        64
19          Milking Stool  3.58  4.03  10.97    271      248    192        77        68
20            Conservator  3.57  3.22  10.46    240      229    184        70        60
21          Lantern House  3.55  2.59   1.78    282      247    198        81        70
22              Furnisher  3.54  3.23   4.36    230      211    177        68        57
23                 Grocer  3.51  2.45   9.54    248      247    209        71        61
24        Charcoal Burner  3.51  2.74   3.35    228      216    172        66        56
25           Writing Desk  3.49  3.09   7.40    262      244    212        70        64
26        Education Bonus  3.47  3.55   6.65    259      238    197        71        63
27          Chimney Sweep  3.44  3.93  12.31    212      204    140        69        51
28       Assistant Tiller  3.43  1.53   4.74    221      219    202        60        53
29            Big Country  3.42  2.99   9.50    267      224    143        80        64
30          Carrot Museum  3.40  2.91   7.20    252      234    172        78        60
31           Field Fences  3.36  3.12  11.78    268      259    204        72        63
32        Potter Ceramics  3.36  3.14   5.57    272      248    220        72        64
33    Half-Timbered House  3.34  3.52  13.33    261      240    138        75        61
34          House Steward  3.32  4.11   5.30    237      216    162        68        55
35              Constable  3.31  3.93   5.94    242      197    142        64        56
36         Wood Collector  3.19  3.72   4.36    224      201    157        56        50
37        Seasonal Worker  3.18  2.11   5.05    238      235    187        62        53
38              Reed Pond  3.17  3.67   8.68    252      232    177        65        56
39              Pet Lover  3.15  2.73   6.08    236      213    182        56        52
40                  Tutor  3.14  4.02   2.72    232      175    156        59        51
41          Clay Supports  3.11  2.18   6.70    257      240    210        68        56
42              Wood Cart  3.10  2.73   8.15    300      291    226        82        65
43    Brushwood Collector  3.09  2.46   4.52    245      236    186        64        53
44          Mining Hammer  3.09  3.85   6.78    268      241    203        66        58
45           Water Worker  3.07  2.71   3.32    242      228    180        63        52
46       Winter Caretaker  3.07  4.57   8.87    251      196    155        59        54
47              Harpooner  3.04  2.37   3.89    230      223    195        58        49
48             Bricklayer  3.03  3.17   5.77    226      210    166        55        48
49         Roof Ballaster  2.97  3.76  12.00    221      208    142        58        46
50         Master Builder  2.97  4.87  13.17    255      191    136        63        53
51           Animal Tamer  2.94  4.27   9.10    238      220    149        60        49
52                 Porter  2.87  3.36   4.39    234      198    153        53        47
53         Moldboard Plow  2.85  2.52   7.89    261      254    237        54        52
54          Animal Dealer  2.83  4.05  10.04    227      209    123        65        45
55        Furniture Maker  2.78  4.10   3.25    231      192    157        55        45
56              Blueprint  2.77  2.62   6.63    263      256    199        58        51
57           Mini Pasture  2.77  3.81   8.55    237      224    196        52        46
58     Skillful Renovator  2.71  4.34   8.94    269      207    140        63        51
59         Private Forest  2.69  3.28   3.33    244      209    158        57        46
60      Master Bricklayer  2.65  4.06   9.15    237      213    124        56        44
61             Feed Fence  2.65  4.31   8.02    270      216    182        57        50
62             Clay Firer  2.64  4.62   9.59    227      189    101        58        42
63        Sleeping Corner  2.63  2.78  10.71    255      239    156        55        47
64            Wood Expert  2.63  3.31   4.88    223      197    156        50        41
65            Clay Supply  2.63  3.94   5.27    277      249    207        64        51
66    Schnapps Distillery  2.62  4.03  11.35    273      237    158        67        50
67         Turnwrest Plow  2.58  3.02   9.17    271      245    214        56        49
68                Bottles  2.54  3.98  10.23    264      227    130        62        47
69         Cross-Cut Wood  2.54  4.79  11.26    259      187    119        64        46
70       Reed-Hatted Toad  2.53  2.95   1.89    243      222    170        59        43
71              Beanfield  2.53  4.57  10.58    277      241    193        61        49
72            Plow Driver  2.51  4.19  10.15    228      165    105        54        40
73            Paper Maker  2.44  4.60   5.89    240      162    117        52        41
74         Recycled Brick  2.44  5.14  10.86    258      173    115        61        44
75               Cottager  2.42  2.62   4.57    260      251    186        57        44
76            Lumber Mill  2.42  3.92   5.74    277      224    157        66        47
77    Straw-Thatched Roof  2.36  3.05   8.12    266      243    168        56        44
78            Foreign Aid  2.36  4.38   5.21    260      151    121        47        43
79       Strawberry Patch  2.36  4.44  12.70    285      254    142        68        47
80           Feed Pellets  2.35  4.78   7.87    274      212    171        52        45
81       Clay Hut Builder  2.33  4.23   7.57    233      205    115        48        38
82               Layabout  2.33  4.40   9.39    251      204    135        58        41
83      Storehouse Keeper  2.30  3.71   4.02    224      182    128        49        36
84             Wood Field  2.30  3.94   8.82    174      149    115        32        28
85               Milk Jug  2.29  4.13   9.95    256      226    185        51        41
86           Game Catcher  2.29  4.86  13.62    237      205    139        52        38
87            Bellfounder  2.28  4.00   6.85    232      168     99        45        37
88                  Hutch  2.27  3.94   7.72    264      232    158        56        42
89          Carriage Trip  2.22  4.92   9.74    270      192    145        52        42
90       Adoptive Parents  2.19  4.65   8.01    241      196    129        51        37
91                  Flail  2.18  4.62   7.10    262      204    163        46        40
92               Stockman  2.18  4.90  11.00    223      173    117        46        34
93     Small-scale Farmer  2.17  3.14   1.73    224      174    153        39        34
94                Trident  2.17  4.37   9.62    289      251    194        56        44
95       Clay Deliveryman  2.15  4.01   5.65    259      223    145        62        39
96            Supply Boat  2.15  4.38   8.32    252      199    163        44        38
97       Small Greenhouse  2.14  4.30   6.79    267      206    135        55        40
98           Clay Carrier  2.14  5.01   7.58    234      167    103        46        35
99            Greengrocer  2.11  3.19   6.92    250      231    191        43        37
100            Plow Maker  2.10  3.60   8.52    265      227    174        47        39
101          Hedge Keeper  2.10  3.83   9.11    252      228    176        44        37
102               Scholar  2.10  4.19   8.72    225      163     98        50        33
103      Wildlife Reserve  2.10  4.42  10.60    259      218    147        48        38
104              Handplow  2.09  3.98   6.06    266      213    153        50        39
105             Carpenter  2.05  2.95   5.07    251      242    179        49        36
106        Organic Farmer  2.05  5.31  13.16    216      159     82        44        31
107      Potato Harvester  2.02  4.51   8.45    240      195    144        47        34
108                 Canoe  2.01  2.81   5.57    256      238    201        48        36
109          Sheep Walker  2.00  4.03   9.78    229      202    126        49        32
110            Farm Store  2.00  4.04   7.71    271      206    137        53        38
111           Stonecutter  1.97  3.77   6.87    246      231    151        50        34
112          Chicken Coop  1.96  3.73   6.27    263      222    154        48        36
113           Wood Cutter  1.90  2.76   2.76    240      226    188        42        32
114          Bread Paddle  1.87  4.70   8.35    268      188    128        50        35
115      Horse-Drawn Boat  1.86  4.35   8.31    246      199    132        50        32
116                 Baker  1.85  4.29   9.77    262      218    133        54        34
117    Schnapps Distiller  1.84  4.04   7.81    241      211    137        44        31
118                Priest  1.84  4.50   6.79    233      155     98        40        30
119          Butter Churn  1.83  4.42   7.31    242      194    137        45        31
120              Claypipe  1.83  4.77   7.49    258      183    128        48        33
121              Pond Hut  1.81  4.03   6.48    268      212    149        54        34
122        Hard Porcelain  1.81  4.61   9.54    277      213    136        47        35
123          Clay Kneader  1.77  4.31   5.77    234      191    141        37        29
124         Acorns Basket  1.77  5.19  11.03    267      188     99        56        33
125    Domestician Expert  1.76  4.92   7.91    260      160     86        50        32
126          Stork's Nest  1.73  4.06   9.56    273      205    108        54        33
127             Childless  1.71  3.07   3.61    226      163    107        42        27
128          Market Crier  1.68  3.76   7.00    221      182    149        28        26
129            Stone Cart  1.61  4.14   6.18    275      211    129        48        31
130            Sheep Well  1.61  5.08  11.63    267      176    102        44        30
131               Plowman  1.60  3.73   3.62    241      187    133        38        27
132               Cowherd  1.59  5.55  10.89    207      129     53        36        23
133               Granary  1.58  4.98   7.95    263      164    105        42        29
134             Puppeteer  1.57  4.16   3.35    228      183    113        39        25
135           Seed Trader  1.57  5.31  10.63    246      118     68        36        27
136        Clay Plasterer  1.56  4.04   7.31    265      217    147        43        29
137         Steam Machine  1.53  4.17   9.99    252      201    120        47        27
138             Junk Room  1.53  4.22   4.76    270      192    164        36        29
139        Truffle Slicer  1.53  4.78  10.47    270      187    117        40        29
140              Thresher  1.51  4.32   7.88    227      164    111        37        24
141             Sack Cart  1.51  4.35   7.02    284      167    104        40        30
142       Threshing Board  1.51  4.77  10.04    275      201    130        45        29
143           Fishing Net  1.50  4.06   7.70    258      207    136        41        27
144  Shifting Cultivation  1.48  4.90   9.73    271      191    140        43        28
145            Millwright  1.46  4.75   8.42    235      165    106        39        24
146                Manger  1.46  5.10  11.87    284      116     67        39        29
147           Stone Tongs  1.45  4.54   8.52    256      189    120        40        26
148  Three-Field Rotation  1.44  4.68  10.71    267      202    144        36        27
149            Corn Scoop  1.43  3.37   6.08    279      237    185        44        28
150        Stone Clearing  1.42  5.25  10.66    251      172    106        41        25
151    Firewood Collector  1.41  4.24   5.37    233      177    114        32        23
152            Consultant  1.40  5.42  12.35    255      176     89        44        25
153                Pastor  1.38  5.08   8.19    228      138     81        36        22
154    Mushroom Collector  1.37  4.07   3.80    240      168    106        32        23
155           Home Brewer  1.30  4.67   8.46    241      185    109        31        22
156         Frame Builder  1.30  4.72   9.06    242      179     98        37        22
157         Pattern Maker  1.30  5.36   9.10    219      119     59        30        20
158         Stable Master  1.27  5.14   8.43    236      152     94        33        21
159         Scythe Worker  1.26  5.00   8.78    249      161     86        36        22
160              Scullery  1.25  3.60   3.87    263      188    127        39        23
161           Herring Pot  1.25  4.19   6.24    262      186    121        35        23
162        Fodder Planter  1.24  5.41  11.62    150       86     39        20        13
163          Thick Forest  1.24  5.42   7.56    266      161     82        34        23
164        Hunting Trophy  1.22  4.82  11.46    175      121     56        34        15
165      Livestock Feeder  1.22  4.85   9.55    245      170     91        38        21
166         Milking Place  1.21  4.68   9.22    284      179    101        37        24
167      Large Greenhouse  1.19  4.39   6.38    289      187    100        40        24
168            Manservant  1.19  4.64   9.00    264      169     86        33        22
169     Resource Recycler  1.18  5.32  10.98    243      149     65        41        20
170                 Lasso  1.16  4.78   9.92    258      193    102        40        21
171             Pitchfork  1.15  4.42   7.09    285      184    116        29        23
172          Stone Carver  1.14  4.92   7.40    250      138     70        33        20
173           Pig Breeder  1.14  5.55  11.94    239      138     63        32        19
174       Building Tycoon  1.12  4.05   5.47    243      172     86        38        19
175      Shepherd's Crook  1.12  4.55   8.32    243      164    119        28        19
176              Lutenist  1.09  5.25   5.41    236      142     81        28        18
177                Stable  1.07  5.26   9.34    254      156    114        24        19
178       Drinking Trough  1.04  5.34  10.24    275      126     72        32        20
179           Hide Farmer  1.02  5.50  13.49    168       96     41        23        12
180             Collector  1.01  4.37   8.81    241      133     72        28        17
181         Cattle Feeder  0.99  4.37   9.28    231      175    101        28        16
182           Cube Cutter  0.96  5.54   9.68    237      117     66        26        16
183           Seed Seller  0.94  4.12   6.41    227      182    126        24        15
184        Clearing Spade  0.93  5.13   8.98    291      165     91        37        19
185              Merchant  0.92  4.68   6.89    249      165     80        34        16
186            Game Trade  0.83  5.40  12.66    259      129     56        35        15
187          Market Stall  0.82  5.58  10.38    260      115     65        23        15
188             Freemason  0.81  4.65   6.55    246      110     56        23        14
189              Conjurer  0.80  4.17   4.79    233      148     87        29        13
190       Oven Firing Boy  0.80  5.02   9.31    231      100     35        19        13
191             Sculpture  0.78  5.22  12.03    275      152     30        36        15
192       Sheep Whisperer  0.77  5.19   5.13    240      108     61        23        13
193           Mantlepiece  0.73  4.65   7.04    255      132     55        32        13
194             Geologist  0.71  4.34   3.42    261      186    104        29        13
195                 Groom  0.69  5.16  10.63    250      125     59        24        12
196            Transactor  0.68  5.23  13.28    230      105     50        26        11
197             Muck Rake  0.67  5.68  11.40    257       92     58        18        12
198      Stable Architect  0.65  5.62  13.39    241       98     51        17        11
199       Clay Embankment  0.62  5.74  10.00    252      123     53        29        11
200          Cattle Buyer  0.56  5.41  10.59    230      135     44        28         9
201   Young Animal Market  0.54  5.80  11.07    293      102     44        27        11
202                 Brook  0.52  4.49   5.97    277      160     89        22        10
203            Stablehand  0.47  5.15  10.04    212      123     56        19         7
204          Wall Builder  0.44  5.07   5.66    229       88     50        13         7
205   Reed Roof Renovator  0.43  5.60   7.82    230      106     45        19         7
206    Interior Decorator  0.41  5.50   5.80    244       78     20        14         7
207           Roughcaster  0.40  5.62   8.24    249       81     37        14         7
208                Basket  0.38  4.80   6.46    265       79     48        12         7
209       Ravenous Hunger  0.34  5.11   9.73    291      118     49        24         7
210           Sugar Baker  0.34  5.40   8.41    251       88     22        16         6
211        Dutch Windmill  0.32  5.32  11.94    270      116     32        15         6
212        Dwelling Mound  0.30  4.89   2.32    286       98     50        13         6
213          Bean Counter  0.25  5.29   5.53    225       85     34        12         4
214      Cattle Whisperer  0.24  5.24   6.35    236      101     34        12         4
215         Wool Blankets  0.21  5.73  13.19    274       49     16        16         4
'''

In [82]:
elo100CDNoASS = '''
                Card Name   PWR   ADP    APR  Deals  Drafted  Plays  Win-Hand  Win-Play
0                Braggart  5.71  1.31  13.45     15       13     11         6         6
1         Begging Student  5.56  1.43   3.44    144      142    133        58        56
2               Bookshelf  5.06  2.08   8.07    158      157    145        56        56
3           Lettuce Patch  4.85  3.10   9.93    162      156    136        57        55
4          Field Watchman  4.55  1.29   3.14    154      154    149        51        49
5                Bookcase  4.36  1.61   4.41    154      153    145        48        47
6                    Loom  4.30  3.22  10.35    156      152    124        51        47
7                Loam Pit  4.25  2.20   8.92    158      158    131        51        47
8               Furnisher  4.23  3.13   4.38    125      121    104        43        37
9               Constable  4.23  3.89   4.94    135      115     84        44        40
10             Cultivator  4.13  2.16   5.22    135      134    117        42        39
11                 Patron  4.12  2.22   2.45    142      138    131        41        41
12          Lantern House  4.11  2.66   1.83    160      143    120        52        46
13             Swing Plow  4.00  2.44   9.21    150      141    125        44        42
14                  Mason  4.00  2.83  12.84    132      120     95        40        37
15           Field Fences  3.94  3.09  11.79    163      158    129        48        45
16              Excavator  3.79  1.96   2.50    147      145    122        46        39
17               Bonehead  3.78  3.04   3.82    140      136    119        42        37
18              Reed Pond  3.72  3.63   8.72    142      129     97        42        37
19    Wooden Hut Extender  3.68  2.49   5.39    132      127    117        37        34
20            Big Country  3.68  3.06   9.41    167      142     85        53        43
21           Writing Desk  3.67  3.11   6.99    144      135    118        39        37
22     Carpenter's Parlor  3.62  1.30   4.54    154      153    141        42        39
23          Chimney Sweep  3.60  3.76  12.18    123      119     79        44        31
24                Caravan  3.57  1.10   5.95     20       20     20         5         5
25          Wood Barterer  3.57  2.78   4.98    136      133    111        36        34
26         Roof Ballaster  3.57  3.39  12.04    124      118     77        39        31
27                 Grocer  3.55  2.37   9.54    145      145    124        40        36
28        Charcoal Burner  3.49  2.65   3.31    139      133    105        40        34
29       Assistant Tiller  3.46  1.48   4.44    128      126    115        36        31
30        Potter Ceramics  3.41  3.11   5.72    159      146    128        44        38
31        Education Bonus  3.38  3.46   6.19    152      138    116        39        36
32            Rammed Clay  3.31  2.11   6.06    151      149    146        36        35
33            Conservator  3.30  3.28  10.31    147      138    106        40        34
34          Carrot Museum  3.23  2.65   7.29    137      132    102        42        31
35          Milking Stool  3.23  4.14  10.90    168      155    115        44        38
36              Blueprint  3.16  2.52   6.71    163      157    125        41        36
37        Seasonal Worker  3.15  2.16   4.75    136      133    100        38        30
38             Wood Field  3.13  3.68   8.51    105       94     72        26        23
39          House Steward  3.13  3.77   4.11    137      124     90        39        30
40    Half-Timbered House  3.12  3.42  13.33    160      144     83        43        35
41             Bricklayer  3.11  3.28   5.66    133      123     94        33        29
42                  Tutor  3.10  4.06   2.28    143      116    104        37        31
43     Skillful Renovator  3.10  4.18   9.00    166      137     94        43        36
44           Water Worker  3.08  2.75   3.32    144      134    105        37        31
45        Furniture Maker  3.08  3.99   3.20    144      119    102        37        31
46       Winter Caretaker  3.05  4.68   8.74    150      119     92        35        32
47         Recycled Brick  3.05  5.07  10.77    150      107     66        45        32
48           Mini Pasture  3.03  4.01   8.57    146      139    124        36        31
49          Mining Hammer  3.02  3.80   6.82    156      142    123        39        33
50           Animal Tamer  3.02  4.23   8.84    137      127     83        35        29
51                 Porter  3.00  3.23   4.18    138      118     90        33        29
52         Wood Collector  2.92  3.58   4.27    137      120     95        33        28
53       Reed-Hatted Toad  2.90  2.94   1.85    138      125    105        33        28
54              Harpooner  2.86  2.58   4.02    150      143    125        37        30
55          Clay Supports  2.82  2.16   6.95    147      135    119        34        29
56        Sleeping Corner  2.80  2.34  10.27    148      140     99        33        29
57               Handplow  2.80  3.88   5.92    148      123     97        33        29
58              Wood Cart  2.78  2.65   8.12    185      179    132        50        36
59               Milk Jug  2.77  3.94   9.83    139      126    102        33        27
60    Brushwood Collector  2.76  2.52   4.49    155      148    113        36        30
61         Private Forest  2.76  3.22   2.90    140      120     97        31        27
62         Cross-Cut Wood  2.74  4.90  11.40    146      108     67        38        28
63            Plow Driver  2.71  4.00   9.97    132      107     68        37        25
64         Master Builder  2.71  4.79  13.16    153      120     75        36        29
65             Clay Firer  2.69  4.66   9.47    133      114     64        33        25
66       Adoptive Parents  2.67  4.53   8.42    134      107     67        32        25
67         Moldboard Plow  2.61  2.45   7.66    159      155    143        30        29
68            Wood Expert  2.61  3.33   4.86    137      126     98        31        25
69            Bellfounder  2.61  3.73   6.52    148      118     73        31        27
70            Clay Supply  2.61  3.96   5.32    159      143    117        36        29
71            Paper Maker  2.60  4.41   5.38    121       85     56        31        22
72           Stork's Nest  2.57  3.55   9.52    167      149     88        43        30
73       Clay Deliveryman  2.57  3.67   5.51    150      135     93        38        27
74            Lumber Mill  2.57  3.79   5.91    167      141    105        39        30
75    Schnapps Distillery  2.55  4.12  11.85    157      138     87        41        28
76               Layabout  2.54  4.32   9.09    152      123     80        35        27
77                  Hutch  2.50  3.61   7.45    154      139    105        33        27
78          Animal Dealer  2.48  4.34   9.58    138      122     64        37        24
79              Childless  2.47  2.65   3.62    133      111     80        33        23
80              Pet Lover  2.46  2.99   6.19    128      116     93        25        22
81             Feed Fence  2.45  4.45   8.39    163      136    115        33        28
82         Turnwrest Plow  2.44  3.08   9.15    158      138    124        29        27
83                Bottles  2.43  3.92   9.85    153      135     79        36        26
84      Master Bricklayer  2.38  4.00   9.20    150      133     75        33        25
85           Feed Pellets  2.37  4.72   7.76    163      132    102        33        27
86       Clay Hut Builder  2.33  4.21   7.56    147      135     75        29        24
87            Greengrocer  2.30  3.11   6.84    143      131    108        27        23
88         Organic Farmer  2.30  5.12  13.12    124       97     43        29        20
89                  Canoe  2.29  3.01   5.65    150      139    117        30        24
90    Straw-Thatched Roof  2.29  3.05   8.15    162      147    102        31        26
91       Potato Harvester  2.29  4.57   8.63    156      132     99        33        25
92              Beanfield  2.27  4.45  10.71    157      142    119        31        25
93     Small-scale Farmer  2.26  3.02   1.67    139      112    100        24        22
94           Clay Kneader  2.23  4.24   5.85    141      115     88        24        22
95      Storehouse Keeper  2.21  3.72   4.21    136      109     73        32        21
96                Scholar  2.21  4.01   8.45    123       88     56        33        19
97       Small Greenhouse  2.19  4.25   6.66    163      130     89        32        25
98            Supply Boat  2.14  4.55   8.31    147      119     95        25        22
99       Wildlife Reserve  2.14  4.56  10.57    147      125     87        27        22
100         Carriage Trip  2.14  4.78   9.65    167      128     99        33        25
101      Strawberry Patch  2.13  4.40  12.80    174      157     85        38        26
102                 Flail  2.12  4.72   7.53    155      120     94        28        23
103             Sack Cart  2.08  4.10   6.69    165      102     67        31        24
104              Stockman  2.08  4.86  10.97    144      116     79        30        21
105              Cottager  2.07  2.61   4.43    152      145    103        30        22
106           Foreign Aid  2.07  4.38   4.82    152       95     71        25        22
107            Plow Maker  2.04  3.53   8.52    154      129     95        25        22
108           Seed Trader  2.03  5.14  10.48    141       78     46        26        20
109    Schnapps Distiller  2.01  3.97   7.32    135      118     76        26        19
110                 Baker  2.01  4.38   9.77    156      128     77        32        22
111               Granary  2.00  4.73   7.81    150      100     67        26        21
112               Plowman  1.98  3.72   3.67    144      112     79        26        20
113              Pond Hut  1.98  4.02   6.33    159      130     94        32        22
114                Priest  1.97  4.50   6.97    152      108     65        30        21
115           Fishing Net  1.92  4.12   7.46    156      129     85        29        21
116          Clay Carrier  1.92  4.88   7.54    134       99     63        26        18
117          Chicken Coop  1.89  3.62   6.23    159      140    100        27        21
118      Horse-Drawn Boat  1.88  4.07   7.88    129      107     73        25        17
119               Trident  1.88  4.36   9.67    182      152    121        32        24
120            Farm Store  1.87  3.93   7.72    168      126     92        28        22
121          Market Crier  1.87  4.00   7.57    122      100     77        18        16
122          Bread Paddle  1.87  4.64   8.34    153       97     64        29        20
123          Game Catcher  1.87  4.85  13.77    145      126     79        32        19
124    Domestician Expert  1.86  4.98   7.54    161      101     50        34        21
125            Sheep Well  1.84  4.94  11.55    155      103     65        27        20
126        Hard Porcelain  1.83  4.67   9.32    164      131     82        32        21
127        Stone Clearing  1.83  5.28  10.67    148      109     69        29        19
128         Acorns Basket  1.81  5.13  10.91    158      112     65        31        20
129             Carpenter  1.79  2.83   5.19    152      147    106        28        19
130           Stonecutter  1.79  3.93   7.12    144      132     81        27        18
131          Butter Churn  1.72  4.38   6.99    133      104     72        22        16
132         Pattern Maker  1.71  5.25   8.76    125       73     38        22        15
133  Three-Field Rotation  1.69  4.74  10.58    161      132     91        27        19
134              Claypipe  1.64  4.80   7.48    148      106     75        24        17
135          Stone Carver  1.63  4.83   7.31    149       93     49        26        17
136             Puppeteer  1.62  4.26   3.61    141      117     66        29        16
137        Clay Plasterer  1.58  4.14   7.26    154      128     82        28        17
138           Stone Tongs  1.56  4.55   8.66    156      109     67        23        17
139             Junk Room  1.55  4.20   4.86    157      112     95        21        17
140            Stone Cart  1.54  4.11   6.13    167      121     83        25        18
141    Firewood Collector  1.54  4.34   5.39    139      105     67        22        15
142                Manger  1.54  4.87  12.05    167       62     37        23        18
143                Pastor  1.54  5.01   7.94    130       83     48        22        14
144          Hedge Keeper  1.52  3.95   8.89    141      123     89        17        15
145  Shifting Cultivation  1.51  4.63   9.31    161      110     81        27        17
146           Herring Pot  1.49  4.51   6.32    153      111     69        24        16
147            Millwright  1.49  4.71   8.30    144      100     60        27        15
148               Cowherd  1.49  5.58  10.73    125       73     33        20        13
149              Scullery  1.48  3.69   3.86    145      107     72        22        15
150        Hunting Trophy  1.47  4.94  11.58    107       80     38        23        11
151          Sheep Walker  1.46  3.97   9.86    137      123     77        22        14
152         Stable Master  1.45  5.14   8.27    128       85     56        19        13
153                Stable  1.42  5.28   9.26    141       97     72        16        14
154          Thick Forest  1.40  5.54   7.75    153       96     48        20        15
155            Corn Scoop  1.39  3.58   5.90    165      139    112        27        16
156      Shepherd's Crook  1.39  4.28   7.90    154      109     81        19        15
157     Resource Recycler  1.36  5.29  11.05    147       98     44        27        14
158            Manservant  1.34  4.63   9.29    160      107     52        23        15
159      Large Greenhouse  1.33  4.37   6.48    172      113     61        24        16
160         Milking Place  1.28  4.60   8.99    178      120     69        28        16
161         Steam Machine  1.26  4.30  10.21    147      117     62        27        13
162           Wood Cutter  1.24  3.06   2.89    138      126    102        19        12
163        Fodder Planter  1.23  5.33  11.42     81       49     24        10         7
164           Home Brewer  1.22  4.58   8.24    140      113     67        18        12
165       Threshing Board  1.21  4.64  10.41    165      125     79        24        14
166        Truffle Slicer  1.20  4.88  10.47    167      108     70        18        14
167       Drinking Trough  1.20  5.25  10.18    166       79     45        23        14
168              Merchant  1.18  4.59   7.03    158      103     58        20        13
169           Hide Farmer  1.18  5.40  13.48     97       63     29        15         8
170          Market Stall  1.18  5.54  10.02    158       72     41        16        13
171         Scythe Worker  1.16  4.78   9.20    148       93     46        19        12
172           Pig Breeder  1.15  5.52  11.76    137       82     33        19        11
173           Seed Seller  1.13  4.07   6.57    126       94     69        15        10
174              Thresher  1.11  4.39   7.78    129       92     63        16        10
175            Consultant  1.10  5.32  12.44    143      102     54        23        11
176        Clearing Spade  1.09  5.14   8.77    183      107     61        24        14
177         Frame Builder  1.08  4.62   8.59    145      105     59        22        11
178                 Lasso  1.06  4.70  10.35    148      112     62        20        11
179             Freemason  1.05  4.36   6.51    150       73     37        17        11
180         Cattle Feeder  1.04  4.47   9.02    138      106     58        16        10
181             Pitchfork  0.96  4.44   7.59    164      103     59        15        11
182              Lutenist  0.96  5.35   5.05    134       83     44        15         9
183    Mushroom Collector  0.95  4.24   4.25    136       89     60        11         9
184             Sculpture  0.91  5.02  11.94    172      103     17        28        11
185       Sheep Whisperer  0.91  5.09   5.19    142       77     48        16         9
186             Collector  0.90  4.26   9.00    143       86     41        16         9
187       Oven Firing Boy  0.88  5.05   9.45    130       58     22        11         8
188       Building Tycoon  0.87  4.07   5.85    148       97     48        18         9
189            Game Trade  0.85  5.28  12.89    152       75     35        22         9
190           Cube Cutter  0.85  5.47   9.26    135       70     38        16         8
191              Conjurer  0.82  3.80   4.33    140       86     51        21         8
192      Livestock Feeder  0.79  4.90   9.98    145      101     51        20         8
193          Cattle Buyer  0.76  5.44  10.93    131       82     30        16         7
194            Transactor  0.75  5.24  13.29    134       66     34        16         7
195             Muck Rake  0.73  5.56  11.46    157       62     39        12         8
196                 Groom  0.70  4.95  10.29    142       76     34        14         7
197       Clay Embankment  0.69  5.82  10.05    144       82     40        17         7
198           Mantlepiece  0.68  4.64   7.03    147       90     34        20         7
199    Interior Decorator  0.63  5.30   5.67    137       50     12        11         6
200           Roughcaster  0.63  5.65   7.58    136       43     19         9         6
201             Geologist  0.60  4.43   3.57    166      115     61        18         7
202            Stablehand  0.60  5.08   9.77    119       63     26        13         5
203      Stable Architect  0.59  5.49  13.53    146       63     34        11         6
204   Reed Roof Renovator  0.48  5.72   7.90    148       76     31        12         5
205                 Brook  0.46  4.33   5.89    156       96     46        14         5
206   Young Animal Market  0.41  5.89  10.89    176       66     28        17         5
207        Dutch Windmill  0.35  5.32  11.68    164       75     25        10         4
208        Dwelling Mound  0.34  5.12   2.29    168       59     31         8         4
209          Bean Counter  0.32  5.02   5.18    136       55     22         6         3
210      Cattle Whisperer  0.32  5.19   5.44    135       58     18         7         3
211         Wool Blankets  0.27  5.56  13.27    160       34     11        10         3
212                Basket  0.26  5.12   6.60    168       49     25         6         3
213       Ravenous Hunger  0.18  5.25   9.50    158       52     20         8         2
214          Wall Builder  0.11  5.21   6.50    135       47     26         3         1
215           Sugar Baker  0.10  5.26   8.09    149       53     11         9         1
'''

In [83]:
elo100CDASS = '''
                Card Name   PWR   ADP    APR  Deals  Drafted  Plays  Win-Hand  Win-Play
0         Begging Student  5.26  1.67   2.95     95       91     88        37        35
1                 Caravan  5.07  1.21   4.47     62       62     59        22        22
2                Braggart  5.00  1.28  13.60     60       60     57        21        21
3                Bonehead  4.87  3.49   4.00     85       83     70        32        29
4          Field Watchman  4.81  1.28   3.71    101      101     97        36        34
5                Bookcase  4.66  2.08   4.71     98       98     90        33        32
6                    Loom  4.29  3.45   9.85    110      104     94        34        33
7           Wood Barterer  4.24  2.53   4.94    101      100     86        34        30
8      Carpenter's Parlor  4.21  1.25   4.32    112      112    107        36        33
9           Milking Stool  4.16  3.84  11.06    103       93     77        33        30
10            Rammed Clay  4.06  1.98   6.16    102      102     94        29        29
11             Cultivator  4.02  2.74   5.06     96       95     88        27        27
12                  Mason  4.00  3.64  12.26    100       94     76        33        28
13            Conservator  3.99  3.12  10.67     93       91     78        30        26
14              Pet Lover  3.97  2.42   5.96    108       97     89        31        30
15    Wooden Hut Extender  3.77  2.43   5.55     91       83     77        26        24
16    Half-Timbered House  3.68  3.68  13.35    101       96     55        32        26
17               Loam Pit  3.67  2.47   8.27    105      105     89        30        27
18    Brushwood Collector  3.65  2.36   4.58     90       88     73        28        23
19         Wood Collector  3.61  3.93   4.48     87       81     62        23        22
20              Wood Cart  3.60  2.86   8.20    115      112     94        32        29
21          Carrot Museum  3.60  3.25   7.07    115      102     70        36        29
22        Education Bonus  3.60  3.69   7.32    107      100     81        32        27
23          House Steward  3.57  4.55   6.78    100       92     72        29        25
24        Charcoal Burner  3.53  2.88   3.40     89       83     67        26        22
25          Clay Supports  3.51  2.20   6.36    110      105     91        34        27
26                 Grocer  3.47  2.57   9.53    103      102     85        31        25
27              Excavator  3.45  1.73   2.66     91       91     80        23        22
28              Harpooner  3.39  2.00   3.67     80       80     70        21        19
29                 Patron  3.39  2.41   2.42    101       94     88        25        24
30       Assistant Tiller  3.38  1.59   5.14     93       93     87        24        22
31          Animal Dealer  3.37  3.64  10.54     89       87     59        28        21
32         Master Builder  3.36  5.00  13.18    102       71     61        27        24
33        Potter Ceramics  3.29  3.18   5.37    113      102     92        28        26
34           Writing Desk  3.27  3.07   7.91    118      109     94        31        27
35          Lettuce Patch  3.26  3.92  10.54     92       87     72        24        21
36        Seasonal Worker  3.22  2.05   5.39    102      102     87        24        23
37         Moldboard Plow  3.22  2.62   8.23    102       99     94        24        23
38                  Tutor  3.21  3.93   3.60     89       59     52        22        20
39          Chimney Sweep  3.21  4.15  12.48     89       85     61        25        20
40          Mining Hammer  3.19  3.92   6.71    112       99     80        27        25
41             Swing Plow  3.16  2.63   9.21    122      113    100        30        27
42      Master Bricklayer  3.12  4.15   9.08     87       80     49        23        19
43              Bookshelf  3.11  2.67   8.46     92       91     82        22        20
44       Winter Caretaker  3.11  4.40   9.06    101       77     63        24        22
45           Water Worker  3.06  2.65   3.31     98       94     75        26        21
46            Big Country  3.00  2.87   9.62    100       82     58        27        21
47           Game Catcher  2.95  4.87  13.42     92       79     60        20        19
48             Feed Fence  2.94  4.06   7.37    107       80     67        24        22
49             Bricklayer  2.92  3.01   5.90     93       87     72        22        19
50               Cottager  2.91  2.64   4.75    108      106     83        27        22
51              Beanfield  2.86  4.75  10.36    120       99     74        30        24
52           Hedge Keeper  2.83  3.69   9.33    111      105     87        27        22
53           Animal Tamer  2.83  4.32   9.42    101       93     66        25        20
54          Lantern House  2.81  2.49   1.71    122      104     78        29        24
55            Wood Cutter  2.80  2.38   2.59    102      100     86        23        20
56           Sheep Walker  2.80  4.14   9.65     92       79     49        27        18
57         Turnwrest Plow  2.78  2.94   9.19    113      107     90        27        22
58            Foreign Aid  2.78  4.38   5.76    108       56     50        22        21
59              Furnisher  2.72  3.37   4.33    105       90     73        25        20
60    Schnapps Distillery  2.71  3.90  10.73    116       99     71        26        22
61                Bottles  2.70  4.07  10.82    111       92     51        26        21
62       Strawberry Patch  2.70  4.49  12.56    111       97     57        30        21
63                 Porter  2.68  3.55   4.70     96       80     63        20        18
64                Trident  2.67  4.40   9.53    107       99     73        24        20
65            Wood Expert  2.66  3.30   4.91     86       71     58        19        16
66            Clay Supply  2.66  3.92   5.19    118      106     90        28        22
67         Private Forest  2.61  3.36   4.02    104       89     61        26        19
68             Clay Firer  2.58  4.57   9.81     94       75     37        25        17
69    Straw-Thatched Roof  2.47  3.04   8.09    104       96     66        25        18
70              Reed Pond  2.47  3.73   8.64    110      103     80        23        19
71              Carpenter  2.45  3.14   4.89     99       95     73        21        17
72           Field Fences  2.45  3.17  11.76    105      101     75        24        18
73      Storehouse Keeper  2.44  3.70   3.76     88       73     55        17        15
74           Clay Carrier  2.43  5.19   7.65    100       68     40        20        17
75        Sleeping Corner  2.40  3.40  11.46    107       99     57        22        18
76          Carriage Trip  2.36  5.20   9.96    103       64     46        19        17
77           Mini Pasture  2.35  3.48   8.51     91       85     72        16        15
78               Stockman  2.35  4.96  11.05     79       57     38        16        13
79       Clay Hut Builder  2.33  4.27   7.60     86       70     40        19        14
80           Feed Pellets  2.32  4.89   8.01    111       80     69        19        18
81        Furniture Maker  2.30  4.29   3.36     87       73     55        18        14
82         Cross-Cut Wood  2.28  4.65  11.08    113       79     52        26        18
83            Paper Maker  2.28  4.82   6.36    119       77     61        21        19
84                  Flail  2.27  4.49   6.51    107       84     69        18        17
85            Stonecutter  2.24  3.57   6.57    102       99     70        23        16
86            Plow Driver  2.23  4.55  10.49     96       58     37        17        15
87             Farm Store  2.22  4.21   7.69    103       80     45        25        16
88            Lumber Mill  2.21  4.14   5.38    110       83     52        27        17
89         Roof Ballaster  2.21  4.24  11.95     97       90     65        19        15
90             Plow Maker  2.19  3.69   8.52    111       98     79        22        17
91            Supply Boat  2.18  4.12   8.34    105       80     68        19        16
92              Blueprint  2.14  2.80   6.49    100       99     74        17        15
93              Constable  2.14  3.99   7.38    107       82     58        20        16
94         Truffle Slicer  2.08  4.63  10.47    103       79     47        22        15
95     Skillful Renovator  2.08  4.64   8.83    103       70     46        20        15
96               Claypipe  2.08  4.71   7.51    110       77     53        24        16
97           Chicken Coop  2.06  3.90   6.35    104       82     54        21        15
98       Small Greenhouse  2.06  4.39   7.02    104       76     46        23        15
99       Reed-Hatted Toad  2.04  2.98   1.95    105       97     65        26        15
100      Wildlife Reserve  2.04  4.23  10.63    112       93     60        21        16
101              Thresher  2.04  4.24   8.02     98       72     48        21        14
102    Small-scale Farmer  2.02  3.37   1.85     85       62     53        15        12
103              Layabout  2.02  4.52   9.84     99       81     55        23        14
104          Butter Churn  1.97  4.46   7.66    109       90     65        23        15
105               Scholar  1.96  4.40   9.10    102       75     42        17        14
106                 Hutch  1.95  4.44   8.26    110       93     53        23        15
107       Threshing Board  1.95  4.97   9.47    110       76     51        21        15
108    Mushroom Collector  1.92  3.89   3.22    104       79     46        21        14
109         Steam Machine  1.90  3.99   9.76    105       84     58        20        14
110           Greengrocer  1.87  3.29   7.02    107      100     83        16        14
111          Bread Paddle  1.86  4.76   8.36    115       91     64        21        15
112      Livestock Feeder  1.86  4.78   9.00    100       69     40        18        13
113      Horse-Drawn Boat  1.83  4.68   8.85    117       92     59        25        15
114            Consultant  1.79  5.55  12.20    112       74     35        21        14
115        Hard Porcelain  1.77  4.51   9.87    113       82     54        15        14
116               Cowherd  1.74  5.52  11.15     82       56     20        16        10
117            Stone Cart  1.72  4.18   6.26    108       90     46        23        13
118              Milk Jug  1.71  4.37  10.10    117      100     83        18        14
119        Organic Farmer  1.71  5.60  13.21     92       62     39        15        11
120           Bellfounder  1.70  4.64   7.77     84       50     26        14        10
121         Acorns Basket  1.70  5.26  11.26    109       76     34        25        13
122                 Canoe  1.62  2.53   5.46    106       99     84        18        12
123    Schnapps Distiller  1.62  4.14   8.43    106       93     61        18        12
124                 Baker  1.62  4.16   9.79    106       90     56        22        12
125         Frame Builder  1.62  4.85   9.77     97       74     39        15        11
126      Adoptive Parents  1.60  4.80   7.56    107       89     62        19        12
127                Priest  1.59  4.49   6.42     81       47     33        10         9
128    Domestician Expert  1.59  4.81   8.42     99       59     36        16        11
129        Recycled Brick  1.59  5.24  10.98    108       66     49        16        12
130              Pond Hut  1.57  4.04   6.73    109       82     55        22        12
131      Clay Deliveryman  1.57  4.55   5.90    109       88     52        24        12
132        Clay Plasterer  1.54  3.89   7.38    111       89     65        15        12
133      Potato Harvester  1.53  4.38   8.07     84       63     45        14         9
134             Junk Room  1.52  4.25   4.62    113       80     69        15        12
135            Corn Scoop  1.50  3.07   6.34    114       98     73        17        12
136       Building Tycoon  1.50  4.01   4.97     95       75     38        20        10
137             Puppeteer  1.48  3.98   2.98     87       66     47        10         9
138          Market Crier  1.44  3.48   6.39     99       82     72        10        10
139  Shifting Cultivation  1.43  5.26  10.31    110       81     59        16        11
140             Pitchfork  1.42  4.41   6.58    121       81     57        14        12
141            Millwright  1.41  4.80   8.57     91       65     46        12         9
142           Home Brewer  1.41  4.82   8.81    101       72     42        13        10
143         Scythe Worker  1.41  5.29   8.30    101       68     40        17        10
144                Manger  1.34  5.37  11.63    117       54     30        16        11
145                 Lasso  1.30  4.89   9.25    110       81     40        20        10
146           Stone Tongs  1.29  4.53   8.36    100       80     53        17         9
147            Sheep Well  1.28  5.27  11.76    112       73     37        17        10
148              Lutenist  1.26  5.12   5.84    102       59     37        13         9
149        Fodder Planter  1.24  5.51  11.93     69       37     15        10         6
150    Firewood Collector  1.22  4.10   5.34     94       72     47        10         8
151              Handplow  1.21  4.11   6.30    118       90     56        17        10
152             Collector  1.17  4.57   8.55     98       47     31        12         8
153                Pastor  1.17  5.18   8.55     98       55     33        14         8
154           Pig Breeder  1.12  5.59  12.13    102       56     30        13         8
155           Cube Cutter  1.12  5.64  10.25    102       47     28        10         8
156          Clay Kneader  1.08  4.42   5.62     93       76     53        13         7
157  Three-Field Rotation  1.08  4.57  10.92    106       70     53         9         8
158         Milking Place  1.08  4.85   9.72    106       59     32         9         8
159         Stable Master  1.06  5.13   8.66    108       67     38        14         8
160            Wood Field  1.04  4.38   9.33     69       55     43         6         5
161               Plowman  1.03  3.73   3.54     97       75     54        12         7
162          Thick Forest  1.01  5.23   7.29    113       65     34        14         8
163               Granary  1.01  5.36   8.21    113       64     38        16         8
164      Large Greenhouse  0.98  4.42   6.23    117       74     39        16         8
165              Scullery  0.97  3.48   3.89    118       81     55        17         8
166            Manservant  0.96  4.66   8.56    104       62     34        10         7
167           Seed Trader  0.95  5.65  10.95    105       40     22        10         7
168           Herring Pot  0.92  3.72   6.13    109       75     52        11         7
169         Cattle Feeder  0.92  4.22   9.63     93       69     43        12         6
170          Wall Builder  0.91  4.90   4.75     94       41     24        10         6
171             Geologist  0.90  4.20   3.21     95       71     43        11         6
172     Resource Recycler  0.89  5.37  10.86     96       51     21        14         6
173           Fishing Net  0.84  3.96   8.10    102       78     51        12         6
174        Hunting Trophy  0.84  4.59  11.22     68       41     18        11         4
175        Stone Clearing  0.83  5.21  10.65    103       63     37        12         6
176            Game Trade  0.80  5.57  12.29    107       54     21        13         6
177           Hide Farmer  0.80  5.70  13.50     71       33     12         8         4
178           Mantlepiece  0.79  4.67   7.05    108       42     21        12         6
179       Drinking Trough  0.79  5.49  10.33    109       47     27         9         6
180              Conjurer  0.77  4.68   5.44     93       62     36         8         5
181         Pattern Maker  0.76  5.54   9.71     94       46     21         8         5
182      Stable Architect  0.75  5.86  13.12     95       35     17         6         5
183   Young Animal Market  0.73  5.64  11.38    117       36     16        10         6
184             Sack Cart  0.72  4.74   7.62    119       65     37         9         6
185           Seed Seller  0.71  4.17   6.23    101       88     57         9         5
186       Oven Firing Boy  0.71  4.98   9.08    101       42     13         8         5
187           Sugar Baker  0.70  5.60   8.73    102       35     11         7         5
188        Clearing Spade  0.66  5.12   9.40    108       58     30        13         5
189                 Groom  0.66  5.49  11.08    108       49     25        10         5
190      Shepherd's Crook  0.64  5.09   9.21     89       55     38         9         4
191                Stable  0.63  5.24   9.48    113       59     42         8         5
192             Childless  0.61  3.98   3.56     93       52     27         9         4
193            Transactor  0.60  5.21  13.25     96       39     16        10         4
194                Basket  0.59  4.27   6.30     97       30     23         6         4
195                 Brook  0.59  4.72   6.05    121       64     43         8         5
196       Sheep Whisperer  0.58  5.45   4.92     98       31     13         7         4
197             Muck Rake  0.57  5.93  11.26    100       30     19         6         4
198             Sculpture  0.55  5.63  12.15    103       49     13         8         4
199       Ravenous Hunger  0.54  5.00   9.90    133       66     29        16         5
200       Clay Embankment  0.53  5.59   9.85    108       41     13        12         4
201              Merchant  0.47  4.84   6.50     91       62     22        14         3
202             Freemason  0.45  5.22   6.63     96       37     19         6         3
203          Stone Carver  0.42  5.11   7.62    101       45     21         7         3
204          Stork's Nest  0.40  5.41   9.70    106       56     20        11         3
205   Reed Roof Renovator  0.35  5.30   7.64     82       30     14         7         2
206            Stablehand  0.31  5.23  10.27     93       60     30         6         2
207          Cattle Buyer  0.29  5.36   9.86     99       53     14        12         2
208          Market Stall  0.28  5.65  11.00    102       43     24         7         2
209        Dutch Windmill  0.27  5.32  12.86    106       41      7         5         2
210        Dwelling Mound  0.24  4.54   2.37    118       39     19         5         2
211          Bean Counter  0.16  5.80   6.17     89       30     12         6         1
212      Cattle Whisperer  0.14  5.30   7.38    101       43     16         5         1
213           Roughcaster  0.13  5.58   8.94    113       38     18         5         1
214    Interior Decorator  0.13  5.86   6.00    107       28      8         3         1
215         Wool Blankets  0.13  6.13  13.00    114       15      5         6         1
'''

In [84]:
elo200CDNew = '''
                Card Name   PWR   ADP    APR  Deals  Drafted  Plays  Win-Hand  Win-Play
0                 Caravan  6.49  1.14   4.64     22       22     22        10        10
1      Carpenter's Parlor  5.52  1.29   4.41     75       75     70        31        29
2                Braggart  5.49  1.08  13.69     13       13     13         5         5
3           Lettuce Patch  5.01  3.24   9.52     77       75     66        28        27
4               Bookshelf  4.97  2.10   8.29     69       69     66        24        24
5          Field Watchman  4.95  1.21   3.45     75       75     71        27        26
6             Conservator  4.62  3.40  10.29     65       62     49        22        21
7                  Patron  4.52  2.25   2.13     60       57     55        19        19
8                Bonehead  4.45  3.27   4.33     61       59     52        20        19
9             Big Country  4.40  2.75   9.03     65       57     36        25        20
10        Begging Student  4.31  1.53   3.49     73       73     70        22        22
11            Rammed Clay  4.23  1.90   5.92     71       69     66        22        21
12               Loam Pit  4.20  2.32   8.64     68       68     61        21        20
13           Writing Desk  4.20  3.00   7.31     68       67     61        20        20
14          Wood Barterer  4.11  2.42   4.80     66       66     61        19        19
15       Reed-Hatted Toad  3.98  2.80   1.95     61       56     43        23        17
16       Winter Caretaker  3.90  4.40   8.77     66       52     43        19        18
17    Brushwood Collector  3.85  2.89   4.63     63       61     46        22        17
18              Furnisher  3.82  2.86   4.69     71       70     62        21        19
19        Furniture Maker  3.79  3.72   3.22     64       53     40        20        17
20          Milking Stool  3.72  3.96  11.37     73       69     54        21        19
21          Animal Dealer  3.70  4.36   9.57     54       53     28        19        14
22                   Loom  3.66  3.38  10.57     82       79     65        23        21
23         Wood Collector  3.64  3.86   4.32     55       51     40        14        14
24              Reed Pond  3.63  3.51   9.04     63       57     45        18        16
25        Charcoal Burner  3.62  2.45   3.53     67       65     53        18        17
26        Potter Ceramics  3.62  3.33   5.16     71       69     61        20        18
27          Mining Hammer  3.57  3.86   6.88     72       66     57        19        18
28            Paper Maker  3.57  4.29   6.42     52       35     26        14        13
29                 Grocer  3.53  2.31   9.22     81       81     72        22        20
30               Bookcase  3.52  1.71   4.71     73       73     66        19        18
31          Lantern House  3.52  2.22   1.63     73       69     59        19        18
32                  Mason  3.52  3.16  12.56     65       62     52        17        16
33             Cultivator  3.51  2.43   5.24     61       61     55        16        15
34         Roof Ballaster  3.40  3.49  11.90     63       61     39        18        15
35             Swing Plow  3.38  2.60   9.15     76       70     59        19        18
36    Wooden Hut Extender  3.34  2.43   5.05     47       44     41        12        11
37              Constable  3.33  4.02   5.61     73       61     44        21        17
38           Water Worker  3.28  3.11   3.67     61       54     43        19        14
39             Clay Firer  3.28  4.51  10.03     61       49     35        14        14
40          House Steward  3.27  4.15   5.13     70       68     54        17        16
41         Recycled Brick  3.24  5.04  10.72     75       55     39        23        17
42              Carpenter  3.17  3.10   5.45     63       61     47        16        14
43         Master Builder  3.13  5.02  13.17     73       55     42        17        16
44         Private Forest  3.11  3.54   3.15     69       57     41        18        15
45          Chimney Sweep  3.02  4.02  12.28     52       51     39        14        11
46       Strawberry Patch  2.94  4.51  12.91     73       69     43        20        15
47            Wood Cutter  2.90  3.12   3.20     64       60     45        19        13
48            Greengrocer  2.90  3.63   8.00     69       59     49        14        14
49     Skillful Renovator  2.90  3.92   9.16     64       49     38        15        13
50           Mini Pasture  2.90  3.98   9.39     64       60     51        15        13
51    Straw-Thatched Roof  2.89  2.91   7.86     79       76     57        19        16
52              Excavator  2.86  1.81   2.57     70       69     61        15        14
53              Pet Lover  2.86  2.67   5.57     65       61     56        14        13
54         Moldboard Plow  2.86  2.84   7.93     65       63     57        13        13
55                  Flail  2.86  4.73   7.55     75       56     42        18        15
56       Small Greenhouse  2.82  4.20   6.52     76       61     46        19        15
57        Education Bonus  2.81  3.73   6.43     61       60     51        15        12
58               Stockman  2.81  5.18  11.50     56       44     28        14        11
59            Foreign Aid  2.78  4.37   4.84     72       46     37        14        14
60        Sleeping Corner  2.77  2.52  10.96     67       65     49        16        13
61           Animal Tamer  2.77  4.52  10.05     67       61     37        17        13
62       Assistant Tiller  2.76  1.50   5.38     62       62     56        14        12
63              Blueprint  2.73  2.78   6.49     68       67     51        14        13
64           Clay Carrier  2.72  5.00   6.73     63       45     26        17        12
65               Milk Jug  2.71  4.28   9.78     58       50     41        14        11
66          Clay Supports  2.69  1.91   6.66     69       68     59        15        13
67          Carrot Museum  2.66  2.64   7.26     59       56     47        12        11
68            Plow Driver  2.66  3.84   9.87     59       49     31        14        11
69    Half-Timbered House  2.65  3.47  13.37     70       66     38        14        13
70                 Priest  2.64  4.58   6.50     65       52     34        13        12
71                  Hutch  2.63  4.15   8.26     76       67     46        16        14
72            Bellfounder  2.62  4.02   6.47     60       47     30        12        11
73        Seasonal Worker  2.60  2.29   5.00     66       66     48        16        12
74              Beanfield  2.60  4.97  10.84     77       72     56        17        14
75              Wood Cart  2.54  2.58   8.28     90       89     74        19        16
76         Turnwrest Plow  2.48  3.18   9.60     75       67     60        15        13
77       Horse-Drawn Boat  2.48  4.44   8.07     75       64     42        17        13
78         Cross-Cut Wood  2.48  4.57  11.26     69       53     39        15        12
79    Schnapps Distillery  2.48  4.61  11.54     69       61     37        19        12
80           Hedge Keeper  2.47  4.40   9.17     81       72     53        18        14
81             Wood Field  2.44  4.13   8.72     41       30     25         7         7
82       Clay Deliveryman  2.38  4.17   6.26     72       66     43        17        12
83      Master Bricklayer  2.38  4.31   9.18     60       55     28        15        10
84             Bricklayer  2.35  3.06   5.29     67       65     49        12        11
85                Bottles  2.35  4.27   9.75     73       62     44        16        12
86     Small-scale Farmer  2.34  3.08   1.49     55       39     37         9         9
87                  Tutor  2.33  4.21   2.66     49       34     29        10         8
88           Market Crier  2.30  4.15   7.30     56       41     33        10         9
89             Plow Maker  2.29  3.59   9.26     75       63     50        13        12
90           Bread Paddle  2.29  4.58   7.73     75       55     37        16        12
91           Feed Pellets  2.29  4.91   8.05     75       55     44        13        12
92           Clay Kneader  2.28  4.47   6.00     69       58     45        12        11
93         Hard Porcelain  2.24  4.59   9.43     70       54     40        11        11
94               Handplow  2.23  4.02   6.02     77       66     48        16        12
95                 Pastor  2.21  5.15   8.18     71       47     33        14        11
96           Field Fences  2.20  2.90  12.20     65       62     51        12        10
97                Scholar  2.20  4.36   8.78     65       50     32        12        10
98              Childless  2.16  2.85   3.26     66       55     39        11        10
99         Clay Plasterer  2.16  4.29   7.03     66       58     36        13        10
100           Lumber Mill  2.14  3.82   5.76     80       60     46        15        12
101           Clay Supply  2.14  3.96   5.41     80       73     61        13        12
102        Fodder Planter  2.13  5.44  11.22     47       25     18         8         7
103               Trident  2.12  4.61   9.63     74       62     52        13        11
104               Plowman  2.10  3.89   4.21     68       55     42        11        10
105                 Canoe  2.07  2.90   5.69     83       78     65        16        12
106     Storehouse Keeper  2.07  3.88   3.74     62       52     35        14         9
107           Seed Trader  2.07  5.56  10.50     62       32     20        11         9
108            Stone Cart  2.06  4.17   6.73     90       69     45        20        13
109          Stork's Nest  2.04  3.89   9.26     70       54     31        15        10
110            Feed Fence  2.04  4.52   8.13     77       63     53        13        11
111         Pattern Maker  2.04  5.27   8.95     56       33     21        10         8
112      Clay Hut Builder  2.01  4.22   7.61     64       58     33        12         9
113      Potato Harvester  2.01  4.78   8.73     78       64     44        17        11
114           Home Brewer  1.96  5.22   8.78     51       37     23         9         7
115            Farm Store  1.95  3.76   6.39     66       50     33        14         9
116       Threshing Board  1.93  5.10   9.53     74       50     36        11        10
117           Stonecutter  1.92  3.98   7.00     67       63     44        11         9
118         Steam Machine  1.89  4.07   9.79     68       57     33        13         9
119              Layabout  1.89  4.38   8.66     68       53     38        12         9
120               Cowherd  1.89  5.85  10.77     53       27     13         9         7
121         Carriage Trip  1.86  5.11  11.00     77       55     37        15        10
122              Pond Hut  1.84  4.19   7.07     70       57     43        14         9
123                Porter  1.79  3.16   4.24     64       55     42        10         8
124         Milking Place  1.79  4.64   8.37     80       56     30        15        10
125           Cube Cutter  1.76  5.39  11.05     65       33     20        11         8
126        Organic Farmer  1.72  5.11  12.83     58       44     24         9         7
127    Mushroom Collector  1.71  3.79   4.66     67       48     32        11         8
128             Harpooner  1.68  2.83   3.50     68       65     54        11         8
129          Thick Forest  1.68  5.84   7.33     68       37     21        11         8
130      Adoptive Parents  1.67  4.70   8.09     60       50     33        11         7
131                 Baker  1.66  4.34   9.93     69       61     41        13         8
132          Butter Churn  1.66  4.40   7.66     69       62     44        10         8
133        Truffle Slicer  1.66  4.81  10.51     86       63     39        11        10
134           Wood Expert  1.64  2.87   4.58     61       53     43         9         7
135          Chicken Coop  1.61  3.68   6.28     71       60     43         9         8
136           Supply Boat  1.61  4.69   8.02     80       68     56        10         9
137           Pig Breeder  1.59  5.70  12.39     72       46     23        13         8
138             Puppeteer  1.57  4.07   3.51     73       57     39        12         8
139              Scullery  1.57  4.27   4.39     73       49     31        15         8
140                 Lasso  1.57  5.00   9.43     73       52     28        12         8
141  Shifting Cultivation  1.57  5.29  10.26     73       49     35        14         8
142          Sheep Walker  1.52  4.05   9.13     66       59     38        12         7
143                Manger  1.48  5.10  11.37     77       29     19        11         8
144             Junk Room  1.47  4.21   4.59     78       53     51         9         8
145      Wildlife Reserve  1.47  4.47  10.48     68       58     40         7         7
146         Acorns Basket  1.47  5.52  10.78     78       52     32        15         8
147           Fishing Net  1.45  4.32   7.30     69       57     37        11         7
148        Clearing Spade  1.45  5.29   8.54     79       45     26        11         8
149          Game Catcher  1.43  5.42  13.61     60       52     41         9         6
150             Pitchfork  1.39  4.86   7.07     72       43     30         8         7
151          Stone Carver  1.39  5.58   7.19     82       43     21        12         8
152    Schnapps Distiller  1.36  3.98   7.65     63       54     34         9         6
153           Stone Tongs  1.34  4.96   8.52     64       49     31         9         6
154              Cottager  1.33  2.62   4.47     75       73     55        11         7
155         Stable Master  1.33  5.37   8.45     75       43     31         9         7
156                Stable  1.30  5.67  10.07     66       39     29         6         6
157            Consultant  1.28  5.29  13.35     78       48     23        12         7
158            Corn Scoop  1.27  3.94   6.00     79       63     43        14         7
159      Large Greenhouse  1.25  4.56   6.48     80       48     29        11         7
160               Granary  1.19  4.93   8.55     60       27     20         5         5
161      Livestock Feeder  1.19  5.26   9.88     72       50     25        11         6
162    Domestician Expert  1.17  4.69   8.78     61       39     18         8         5
163         Scythe Worker  1.17  5.17  10.05     61       36     19         8         5
164           Hide Farmer  1.14  5.48  13.42     50       31     12         7         4
165       Oven Firing Boy  1.13  5.43   9.33     63       21     12         6         5
166      Shepherd's Crook  1.12  4.93   7.85     64       44     33         8         5
167         Cattle Feeder  1.08  4.94   9.66     66       52     29        11         5
168            Transactor  1.07  5.59  13.56     67       34     18        10         5
169            Sheep Well  1.06  5.22  11.97     81       49     29        11         6
170              Merchant  1.05  4.37   6.40     68       43     20        13         5
171            Manservant  1.05  4.49   8.80     68       47     25         8         5
172     Resource Recycler  1.05  5.27  10.30     68       41     20        12         5
173             Sculpture  1.03  5.20  12.00     83       49     11        16         6
174             Collector  1.02  4.40   8.36     70       45     22        10         5
175       Building Tycoon  1.01  3.98   5.07     71       47     27        10         5
176        Stone Clearing  1.01  5.51  10.88     71       47     25        12         5
177            Millwright  1.00  4.86   8.08     57       36     25         8         4
178    Firewood Collector  0.99  4.04   4.00     72       49     33         8         5
179  Three-Field Rotation  0.97  4.79  10.98     74       61     42         8         5
180       Clay Embankment  0.90  5.92  10.35     79       36     17         7         5
181        Hunting Trophy  0.89  4.45  11.72     48       31     18         7         3
182           Seed Seller  0.85  4.66   7.03     67       53     36         6         4
183           Herring Pot  0.84  4.70   6.53     68       53     36         7         4
184              Claypipe  0.83  4.73   7.64     69       51     39         7         4
185         Frame Builder  0.79  4.98   8.50     72       52     28         8         4
186           Roughcaster  0.74  5.35   8.92     58       23     12         6         3
187              Conjurer  0.68  4.34   5.41     63       35     22         8         3
188          Cattle Buyer  0.68  5.56  12.10     63       34     10         7         3
189      Stable Architect  0.68  6.23  13.20     63       30     15         5         3
190           Mantlepiece  0.67  4.19   7.05     64       32     19         6         3
191       Drinking Trough  0.65  5.83  10.74     88       35     19         7         4
192                 Groom  0.64  4.78  11.21     67       32     14         8         3
193            Stablehand  0.63  5.76  10.38     68       29      8         9         3
194             Freemason  0.58  4.78   7.25     74       32     16         5         3
195            Game Trade  0.57  5.68  12.43     75       31     14         8         3
196             Sack Cart  0.56  4.35   7.14     77       43     28         6         3
197        Dwelling Mound  0.54  4.66   2.35     80       35     20         5         3
198                Basket  0.54  5.42   7.20     79       19     10         5         3
199   Young Animal Market  0.49  6.28  12.23     88       29     13         9         3
200      Cattle Whisperer  0.48  5.55   6.40     59       20      5         5         2
201              Thresher  0.45  4.60   8.44     63       47     34         5         2
202          Market Stall  0.42  5.83  10.15     68       24     13         3         2
203    Interior Decorator  0.41  5.42   7.00     69       19      5         4         2
204                 Brook  0.36  4.92   6.77     80       38     22         6         2
205          Bean Counter  0.26  5.50   6.75     56       18      8         2         1
206       Sheep Whisperer  0.24  5.74   5.82     59       19     11         4         1
207   Reed Roof Renovator  0.23  5.96   7.36     63       25     11         4         1
208             Muck Rake  0.23  6.04  11.85     63       24     13         3         1
209         Wool Blankets  0.21  5.83  13.00     68        6      4         3         1
210           Sugar Baker  0.21  6.10   8.50     67       20      6         4         1
211        Dutch Windmill  0.20  5.23  11.50     71       26      8         2         1
212       Ravenous Hunger  0.19  5.46  10.43     76       24      7         7         1
213             Geologist  0.00  5.04   2.33     69       48     27         4         0
214          Wall Builder  0.00  5.60   5.40     74       20     10         1         0
215              Lutenist  0.00  5.96   5.73     55       26     15         2         0
'''

In [100]:
writer = pd.ExcelWriter('./result2.xlsx', engine='openpyxl')
# book = load_workbook(writer.path)
# writer.book = book
# df.to_excel(excel_writer=writer, sheet_name='sheet3')
# writer.save()
raw_to_df(all_arena_games).to_excel(excel_writer=writer, sheet_name='所有竞技场对局', index=False)

raw_to_df(elo200_games).to_excel(excel_writer=writer, sheet_name='elo>=200竞技场', index=False)

raw_to_df(incCD).to_excel(excel_writer=writer, sheet_name='所有含CD对局', index=False)

raw_to_df(elo200CD).to_excel(excel_writer=writer, sheet_name='elo>=200含CD对局', index=False)

raw_to_df(elo100CD).to_excel(excel_writer=writer, sheet_name='elo>100含CD对局', index=False)

raw_to_df(elo200CDNew).to_excel(excel_writer=writer, sheet_name='elo>=200含CD对局新', index=False)

writer.save()